In [ ]:
from IPython.display import HTML, display

HTML('''<script>
code_show=true;
function code_toggle() {
  if (code_show){
  $('div.input').hide();
  } else {
  $('div.input').show();
  }
  code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
<input type="submit" value="Reveal/Hide Code"></form>''')

In [ ]:
# # Environment creation commands:
# conda create -y -n py3 python=3.7
# conda activate py3
# conda install -y nb_conda
# pip install altair altair_saver
# pip install wget
# pip install altair vega_datasets altair_saver
# pip install matplotlib seaborn datetime
# pip install python-pptx
# pip install pycountry_convert
# wget https://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_mac64.zip
# unzip chromedriver_mac64.zip
# rm chromedriver_mac64.zip
# mv chromedriver /usr/local/bin/

# jupyter notebook covid.ipynb

# # Environment removal after the test
# conda deactivate py3
# conda env remove -y -n py3

# # Current issues:
# altair chart save produces a blank image (svg or png) for vconcat of hconcat charts
# https://github.com/altair-viz/altair/issues/2253

In [ ]:
# Package imports
import os
import re

import altair as alt
from altair_saver import save
from vega_datasets import data

import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import datetime

import pycountry_convert
from pycountry_convert import country_alpha2_to_continent_code

import requests
from urllib.request import urlopen

# Package configurations
alt.data_transformers.disable_max_rows()


# Configuration for notebook
aggregate_states = True # option to roll-up multiple states to a top level per country view
show_charts_for_all_countries = False # option to show full long list of charts for all countries

sort_by_confirmed = False # sort by confirmed cases or deaths

include_uk_local_authorities = False # takes quite a while to download

num_cases_threshold = 1500 # optional number of cases threshold to filter out what would otherwise be 'noisy' statistics; value is None, or integer threashold

countries_to_aggregate = ['Australia', 'Canada', 'China']

run_date_str = "{:%d-%b-%Y}".format(datetime.datetime.now())

# Slide export configuration
export_slides = True
slides_file_name = 'COVID_Summary.pptx'
template_pptx = 'widescreen.pptx' # input slide template, to help set aspect ratio
saved_image_name = 'image.png'

In [ ]:
def display_h1(myString):
    """display as heading 1"""

    display(HTML('<h1>%s</h1>' % myString))

def display_h2(myString):
    """display as heading 2"""

    display(HTML('<h2>%s</h2>' % myString))

def display_table(myDF):
    """display as formatted table"""

    total_id = 'totalID'
    header_id = 'headerID'
    data_in_html = myDF.to_html(index=False)
    tr_style = '<style>.dataframe td { text-align: left; }</style>'
    style_in_html = """<style>
        table#{total_table} {{color='black';font-size:13px; text-align:center; border:0.2px solid black;
                             border-collapse:collapse; max-width: 900px; word-wrap:break-word; white-space:pre-wrap; height='250'; }}
        thead#{header_table} {{background-color: #4D4D4D; color:#ffffff}}
        </style>""".format(total_table=total_id, header_table=header_id)
    data_in_html = re.sub(r'<table',
                          r'<table id=%s ' % total_id, data_in_html)
    data_in_html = re.sub(r'<thead', r'<thead id=%s ' % header_id, data_in_html)

    display(HTML(style_in_html + tr_style + data_in_html))

def display_set_full_width():
    """use the full screen width"""

    display(HTML("""
    <style>
        div#notebook-container    { width: 100%; }
        div#menubar-container     { width: 65%; }
        div#maintoolbar-container { width: 99%; }
    </style>
    """))

# load country information
# https://github.com/CSSEGISandData/COVID-19
def load_country_data():
    base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'
    file_name = f'UID_ISO_FIPS_LookUp_Table.csv'
    url = os.path.join(base_url, file_name)
    df = pd.read_csv(url)

    return df

def get_continent_from_iso2_code(iso2):
    try:
        continent = country_alpha2_to_continent_code(iso2)
    except:
        continent = 'Unknown'
        
    return continent

def add_continent(df):
    df_countries = load_country_data()
    df_countries = df_countries.filter(['Country_Region', 'Province_State', 'Combined_Key', 'iso2', 'iso3', 'Population'])

    # fix missing space in Johns Hopkins online data before merge
    combined_key_replace_dict = {'Northwest Territories,Canada': 'Northwest Territories, Canada'}
    df_countries['Combined_Key'] = df_countries['Combined_Key'].replace(combined_key_replace_dict)
    
    df = df.merge(df_countries, how='left', on='Combined_Key')
    df = df.drop_duplicates()

    df['continent'] = df['iso2'].apply(lambda x: get_continent_from_iso2_code(x))

    df.loc[df['iso2']=='SX', 'continent'] = 'NA' # add missing lookup for Sint Marten (Dutch part)
    df.loc[df['Combined_Key']=='Namibia', 'continent'] = 'AF' # add missing lookup for Namibia
    
    continent_dict = {'AN':'Antarctica', 'AS':'Asia', 'EU':'Europe', 'SA':'South America', 'NA':'North America', 'OC':'Oceania', 'AF':'Africa'}
    df['continent'] = df['continent'].replace(continent_dict)

    return df

# load data series
# - valid 'name' options are: 'confirmed', 'deaths', or 'recovered'
# - valid 'selection' options are: 'global' or 'US'
# https://github.com/CSSEGISandData/COVID-19
def load_individual_timeseries(name, selection='global', agg_states=True, keep_latlong=True):
    base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series'
    file_name = f'time_series_covid19_{name}_{selection}.csv'
    
#     print(file_name)
    if selection == 'global':
        index_col_list = ['Country/Region', 'Province/State', 'Lat', 'Long']
    else: # 'US'
        if name == 'confirmed':
            index_col_list = ['Country_Region', 'Province_State', 'Lat', 'Long_', 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key']
        else: # 'deaths'
            index_col_list = ['Country_Region', 'Province_State', 'Lat', 'Long_', 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population']
#         index_col_list = ['Country_Region', 'Province_State', 'Lat', 'Long_', 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population']

    url = os.path.join(base_url, file_name)
    df = pd.read_csv(url, index_col=index_col_list)
    df['type'] = name.lower()
    df.columns.name = 'date'
    
    df = df.set_index('type', append=True)
    if not keep_latlong:
        df = df.reset_index(['Lat', 'Long'], drop=True)
        column_list = ['country', 'state', 'type', 'cases']
    else:
        if selection == 'US':
            if name == 'confirmed':
                columns_to_drop = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Combined_Key']
            else:
                columns_to_drop = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Combined_Key', 'Population']

            df = df.reset_index(columns_to_drop, drop=True)

            column_list = ['country', 'state', 'lat', 'long', 'admin2', 'type', 'cases']

        else:
            column_list = ['country', 'state', 'lat', 'long', 'type', 'cases']

    df = df.stack()
    df = df.reset_index()
    df = df.set_index('date')
         
    df.index = pd.to_datetime(df.index)
    df.columns = column_list
    
    df['cases'] = df['cases'].astype(int)

    # Aggregate large countries split by states
    if agg_states:
        if selection == 'global':
            df_non_states = df[~df['country'].isin(countries_to_aggregate)]
            df_states     = df[ df['country'].isin(countries_to_aggregate)]
            df_states     = df_states.groupby(['country', 'date', 'type']).agg({'lat':'mean', 'long':'mean', 'cases':'sum'})
            df_states     = df_states.reset_index(['country', 'type'])
            df = pd.concat([df_non_states, df_states])
        else: # 'US'
            df = df.groupby(['country', 'state', 'date', 'type']).agg({'lat':'mean', 'long':'mean', 'cases':'sum'})
        
        df = df.reset_index()
        df = df.set_index('date')

    countries_to_drop = ['Diamond Princess'] # drop the cruise ship
    df = df[~df['country'].isin(countries_to_drop)]
    
    return df

# - valid 'selection' options are: 'global' or 'US'
def load_covid_data(selection='global', agg_states=False, p_crit=.05, filter_n_days_100=None, keep_latlong=True):
    df = load_individual_timeseries('confirmed', selection=selection, agg_states=agg_states, keep_latlong=keep_latlong)
    df = df.rename(columns={'cases': 'confirmed'})

    df['state'] = df['state'].fillna('')

    # Estimated critical cases
    df = df.assign(critical_estimate=df.confirmed*p_crit)

    # Compute days relative to when 100 confirmed cases was crossed
    df.loc[:, 'days_since_100'] = np.nan
    for country in df.country.unique():
        if not df.loc[(df.country == country), 'state'].isnull().all():
            for state in df.loc[(df.country == country), 'state'].unique():
                df.loc[(df.country == country) & (df.state == state), 'days_since_100'] = \
                    np.arange(-len(df.loc[(df.country == country) & (df.state == state) & (df.confirmed < 100)]), 
                              len(df.loc[(df.country == country) & (df.state == state) & (df.confirmed >= 100)]))
        else:
            df.loc[(df.country == country), 'days_since_100'] = \
                np.arange(-len(df.loc[(df.country == country) & (df.confirmed < 100)]), 
                          len(df.loc[(df.country == country) & (df.confirmed >= 100)]))

    # Add recovered cases
#     df_recovered = load_individual_timeseries('Recovered')
#     df_r = df_recovered.set_index(['country', 'state'], append=True)[['cases']]
#     df_r.columns = ['recovered']

    # Add deaths
    df_deaths = load_individual_timeseries('deaths', selection=selection)
    df_d = df_deaths.set_index(['country', 'state'], append=True)[['cases']]
    df_d.columns = ['deaths']

    df = (df.set_index(['country', 'state'], append=True)
#             .join(df_r)
            .join(df_d)
            .reset_index(['country', 'state'])
    )
    
    if filter_n_days_100 is not None:
        # Select countries for which we have at least some information
        countries = pd.Series(df.loc[df.days_since_100 >= filter_n_days_100].country.unique())
        df = df.loc[lambda x: x.country.isin(countries)]

    df['Combined_Key'] = (df['state'] + ", " + df['country']).str.strip(', ')

    df = df.reset_index()

    df = add_continent(df) # consider to add continent for time series data too
    
    df['deaths_per_M_population']    = df['deaths']    / df['Population'] * 1E6 # per 1M population
    df['confirmed_per_M_population'] = df['confirmed'] / df['Population'] * 1E6 # per 1M population

    return df

# - valid 'selection' options are: 'global' or 'US'
def load_covid_data_us(agg_states=True, keep_latlong=True):
    selection = 'US'
    df = load_individual_timeseries('confirmed', selection=selection, agg_states=agg_states, keep_latlong=keep_latlong)
    df = df.rename(columns={'cases': 'confirmed'})

    df['state'] = df['state'].fillna('')
    if not agg_states:
        df['admin2'] = df['admin2'].fillna('')
        index_col_list = ['country', 'state', 'admin2']
    else:
        index_col_list = ['country', 'state']
        
    # Add deaths
    df_deaths = load_individual_timeseries('deaths', selection=selection)
    df_d = df_deaths.set_index(index_col_list, append=True)[['cases']]
    df_d.columns = ['deaths']

    df = (df.set_index(index_col_list, append=True)
            .join(df_d)
            .reset_index(index_col_list)
    )
    
    df = df.reset_index()

    return df

def get_totals_from_time_series(df):
    df_totals = df.copy().groupby(['country', 'state']).last()
    df_totals = df_totals.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    return df_totals

def get_continent_totals(df):
    df_totals = df.copy().drop(['country', 'state', 'critical_estimate', 'days_since_100', 'type'], axis=1)
    df_totals = df_totals.groupby(['continent']).agg({'lat':'mean', 'long':'mean', 'confirmed':'sum', 'deaths':'sum', 'Population':'sum'})
    df_totals = df_totals.reset_index()
    
    df_totals['deaths_per_M_population']    = df_totals['deaths']    / df_totals['Population'] * 1E6 # per 1M population
    df_totals['confirmed_per_M_population'] = df_totals['confirmed'] / df_totals['Population'] * 1E6 # per 1M population

    return df_totals

In [ ]:
display_set_full_width()
display_h1('COVID Data Visualisation')
display_h2('Load COVID data')
df_covid_time_series = load_covid_data(agg_states=aggregate_states, filter_n_days_100=1, keep_latlong=True)
df_totals = get_totals_from_time_series(df_covid_time_series)

df_covid_time_series_us = load_covid_data_us()
df_totals_us = get_totals_from_time_series(df_covid_time_series_us)

# optionally filter by num cases, to exclude what would otherwise be 'noisy' statistics
if num_cases_threshold:
    selected_list = df_totals.loc[df_totals['confirmed']>=num_cases_threshold, 'Combined_Key']
    exclude_list  = df_totals.loc[df_totals['confirmed']<num_cases_threshold, 'Combined_Key']
    print('Excluding cases <', num_cases_threshold)
    print('- list:', list(exclude_list))
    df_totals = df_totals[df_totals['Combined_Key'].isin(selected_list)]
    df_covid_time_series = df_covid_time_series[df_covid_time_series['Combined_Key'].isin(selected_list)]
    
df_unknown_continents = df_totals[df_totals['continent']=='Unknown']
if len(df_unknown_continents):
    print('Unknown continents:')
    display(df_totals[df_totals['continent']=='Unknown'])

df_country_continent = df_totals.copy().filter(['continent', 'Combined_Key', 'country', 'state']).drop_duplicates()

latest_date = df_covid_time_series['date'].max()
latest_date_str = "{:%d-%b-%Y}".format(latest_date)
print('Most recent data: ', latest_date_str)

In [ ]:
# https://coronavirus.data.gov.uk
# https://coronavirus.data.gov.uk/developers-guide

from requests import get
from json import dumps
from http import HTTPStatus

def df_from_uk_gov(area_type = 'overview', area_name = None, latest=False, timeout=200):
    ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"
    cases_field = 'cumCasesByPublishDate' if area_type == 'nation' else 'cumCasesBySpecimenDate'
    if area_type in ['overview', 'nation']:
        deaths_field = 'cumDeaths28DaysByPublishDate' # Aug-2020: UK Government changed deaths metric from 'cumDeathsByDeathDate'
    else:
        deaths_field = 'cumDeaths28DaysByDeathDate' # Aug-2020: UK Government changed deaths metric from 'cumDeathsByDeathDate'
    
    # areaType: Area type as string
    # areaName: Area name as string
    # areaCode: Area Code as string
    # date: Date as string [YYYY-MM-DD]
    filters = [
        f"areaType={ area_type }" # overview, nation, region, nhsRegion, utla, ltla
    ]
    if area_name: filters = filters + [f"areaName={ area_name }"]

    # Some metrics are not available for specific areaType values.
    # For instance, we have newCasesByPublishDate and cumCasesByPublishDate only available for areaType=nation but not for region,utla, or ltla.
    # Conversely, we have newCasesBySpecimenDate and cumCasesBySpecimenDate available for region, utla, and ltla but not for nation.
    structure = {
        "date": "date",
        "type": "areaType",
        "name": "areaName",
        "code": "areaCode",
        "cases": cases_field,
        "deaths": deaths_field
    }

    api_params = {
        "filters": str.join(";", filters),
        "structure": dumps(structure, separators=(",", ":"))
    }
    if latest: api_params["latestBy"] = deaths_field # must be a metrics field included in 'structure'

    page_number = 1
    df_overall = pd.DataFrame() # initial value
    finished = False
    while not finished:
        # Adding page number to query params
        api_params["page"] = page_number
        
        response = get(ENDPOINT, params=api_params, timeout=timeout)

        if response.status_code >= HTTPStatus.BAD_REQUEST:
            raise RuntimeError(f'Request failed: { response.text }')
        elif response.status_code == HTTPStatus.NO_CONTENT:
            print('No data returned from query')
            break

        df = pd.DataFrame() # initial value
        data = response.json()
        # {'length': 216, 'maxPageLimit': 1000, 'data': [{'date': '2020-08-07', 'name': 'England', 'code': 'E92000001', 'cases': {'daily': 796, 'cumulative': 266645}, 'deaths': {'daily': None, 'cumulative': None}}, {'date': '2020-08-06', 'name': 'England', 'code': 'E92000001', 'cases': {'daily': 826, 'cumulative': 265849}, 'deaths': {'daily': 14, 'cumulative': 41886}}, 
        df = pd.DataFrame.from_dict(data['data'])
        df_overall = pd.concat([df_overall, df])
        
        # check if more to read
        length = data['length']
        page_limit = data['maxPageLimit']
        if (length < page_limit) or (page_limit == 1):
            finished = True
        else:
            page_number += 1
        
    return df_overall

# get UK population for specified area codes
def df_get_uk_population(area_code_list):
    # UK population - mid 2019
    # Could not find a useable online source, so downloaded and extracted to CSV file
    # https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fpopulationestimatesforukenglandandwalesscotlandandnorthernireland%2fmid2019april2020localauthoritydistrictcodes/ukmidyearestimates20192020ladcodes.xls
    # Extracted subset of data to local CSV file
    uk_population_file_2019 = 'UK_Population_Mid_2019.csv'
    df_uk_population_2019 = pd.read_csv(uk_population_file_2019, thousands=',')

    # In April 2020, the district authorities and county council of Buckinghamshire merged to make a single local authority.
    # https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/methodologies/methodologyguideformid2015ukpopulationestimatesenglandandwalesjune2016

    # https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fpopulationestimatesforukenglandandwalesscotlandandnorthernireland%2fmid20182019laboundaries/ukmidyearestimates20182019ladcodes.xls
    # Extracted subset of data to local CSV file
    uk_population_file_2018 = 'UK_Population_Mid_2018.csv'
    df_uk_population_2018 = pd.read_csv(uk_population_file_2018, thousands=',')

    area_codes_in_population_data_2019 = df_uk_population_2019['Area code'].unique()
    area_codes_missing_in_population_data_2019 = list(set(area_code_list) - set(area_codes_in_population_data_2019))
    if len(area_codes_missing_in_population_data_2019):
        df_uk_population_insert_from_2018 = df_uk_population_2018[df_uk_population_2018['Area code'].isin(area_codes_missing_in_population_data_2019)]
        df_uk_population = pd.concat([df_uk_population_2019, df_uk_population_insert_from_2018])
    else:
        df_uk_population = df_uk_population_2019
        
    return df_uk_population
    
def df_from_json_file(url):
    data = urlopen(url).read()
    raw_json = json.loads(data)

    formatted_json = [feature['attributes'] for feature in raw_json['features']]
    df = pd.DataFrame(formatted_json)

    return df

def add_region_mapping_uk(df):
    df = df.copy()
    
    # Local Authority District to Region (April 2019) Lookup in England
    # https://geoportal.statistics.gov.uk/datasets/local-authority-district-to-region-april-2019-lookup-in-england/geoservice
    #  LAD19CD    LAD19NM    RGN19CD    RGN19NM    FID  
    url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LAD19_RGN19_EN_LU/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
    columns_to_drop = ['FID']
    df_region_lookup = df_from_json_file(url)
    df_region_lookup = df_region_lookup.drop(columns_to_drop, axis=1)

    # Lower Tier Local Authority to Upper Tier Local Authority (April 2019) Lookup in England and Wales
    #  LTLA19CD    LTLA19NM    UTLA19CD    UTLA19NM    FID  
    url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LTLA19_UTLA19_EW_LU/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
    df_authority_lookup = df_from_json_file(url)
    columns_to_drop = ['FID']
    df_authority_lookup = df_authority_lookup.drop(columns_to_drop, axis=1)
    df_authority_lookup = df_authority_lookup.drop_duplicates()
    
    local_authority_tiers_to_regions = df_authority_lookup.merge(df_region_lookup, how='left', left_on='LTLA19CD', right_on='LAD19CD')

    # Ward to Local Authority District to County to Region to Country (December 2019) Lookup in United Kingdom
    # https://geoportal.statistics.gov.uk/datasets/ward-to-local-authority-district-to-county-to-region-to-country-december-2019-lookup-in-united-kingdom/geoservice
    #  WD19CD    WD19NM    LAD19CD    LAD19NM    CTY19CD    CTY19NM  RGN19CD    RGN19NM    CTRY19CD    CTRY19NM    FID  
    url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/WD19_LAD19_CTY19_OTH_UK_LU/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
    df_county_lookup = df_from_json_file(url)
    columns_to_filter = ['LAD19CD', 'LAD19NM', 'CTY19CD', 'CTY19NM', 'RGN19CD', 'RGN19NM', 'CTRY19CD', 'CTRY19NM']
    df_county_lookup = df_county_lookup.filter(columns_to_filter)
    df_county_lookup = df_county_lookup.drop_duplicates()

    columns_to_filter = ['RGN19CD', 'CTRY19CD', 'CTRY19NM']
    region_to_country = df_county_lookup.filter(columns_to_filter)
    region_to_country = region_to_country.drop_duplicates()
    local_authority_tiers_to_regions_to_countries = local_authority_tiers_to_regions.merge(region_to_country, how='left', on='RGN19CD')

    # Create dictionaries to map field values
    LTLA19CD_to_RGN19NM_dict = dict(set(zip(local_authority_tiers_to_regions_to_countries['LTLA19CD'], local_authority_tiers_to_regions_to_countries['RGN19NM'])))
    UTLA19CD_to_RGN19NM_dict = dict(set(zip(local_authority_tiers_to_regions_to_countries['UTLA19CD'], local_authority_tiers_to_regions_to_countries['RGN19NM'])))
    RGN19CD_to_CTRY19NM_dict = dict(set(zip(local_authority_tiers_to_regions_to_countries['RGN19CD'], local_authority_tiers_to_regions_to_countries['CTRY19NM'])))
    RGN19NM_to_CTRY19NM_dict = dict(set(zip(local_authority_tiers_to_regions_to_countries['RGN19NM'], local_authority_tiers_to_regions_to_countries['CTRY19NM'])))
    
    # Lookup location hierarchy
    df.loc[df['Area type']=='Region', 'RegionName'] = df['Area name']
    df.loc[df['Area type']=='Upper tier local authority', 'RegionName'] = df['Area code'].map(UTLA19CD_to_RGN19NM_dict)
    df.loc[df['Area type']=='Lower tier local authority', 'RegionName'] = df['Area code'].map(LTLA19CD_to_RGN19NM_dict)
    df['CountryName'] = df['RegionName'].map(RGN19NM_to_CTRY19NM_dict)
    df.loc[df['Area type']=='Nation', 'CountryName'] = df['Area name']

    return df

def df_merge_uk_population(df):
    area_code_list = df['Area code'].unique()
    df_uk_population = df_get_uk_population(area_code_list)

    df = df.merge(df_uk_population.filter(['Area code', 'Population']), how='left', on='Area code')

    return df

def load_covid_data_uk(area_type='all', latest=False):
    # loading is slow, so print something on screen to help show progress
    if area_type == 'all':
        print('UK data - loading:', area_type)
        print('Nations')
        df_uk_nations = df_from_uk_gov(area_type = 'nation', area_name = None, latest=latest)
        print('- length: ', len(df_uk_nations))

        print('Regions')
        df_uk_regions = df_from_uk_gov(area_type = 'region', area_name = None, latest=latest)
        print('- length: ', len(df_uk_regions))

        if not include_uk_local_authorities:
            df_uk_utla = pd.DataFrame()
            df_uk_ltla = pd.DataFrame()
        else:
            print('Upper tier local authorities')
            df_uk_utla = df_from_uk_gov(area_type = 'utla', area_name = None, latest=latest)
            print('- length: ', len(df_uk_utla))

            print('Lower tier local authorities')
            df_uk_ltla = df_from_uk_gov(area_type = 'ltla', area_name = None, latest=latest)
            print('length: ', len(df_uk_ltla))

        df = pd.concat([df_uk_nations, df_uk_regions, df_uk_utla, df_uk_ltla], ignore_index=True)

    else:
        print('UK data - loading:', area_type)
        df = df_from_uk_gov(area_type = area_type, area_name = None, latest=latest)
        print('- length: ', len(df))
    
    df = df.rename(columns={'name':'Area name', 'code':'Area code', 'type':'Area type', 'cases': 'confirmed'})

    df['Area type'] = df['Area type'].map({'nation':'Nation', 'region':'Region', 'utla':'Upper tier local authority', 'ltla':'Lower tier local authority'})

    # Add region and population information
    df = add_region_mapping_uk(df)
    df = df_merge_uk_population(df)
        
    # Add normalised data
    df['confirmedNorm'] = df['confirmed']/(df['Population']/100E3) # cases normalised per 100k population
    df['deathsNorm'] = df['deaths']/(df['Population']/100E3) # deaths normalised per 100k population

    # Sort by date to help with later delta calculations
    df = df.sort_values(by=['date'], ascending=True)

    return df

# Load all UK data as combined data-frame
df_time_series_uk = load_covid_data_uk(latest=False)
df_time_series_england = df_time_series_uk[df_time_series_uk['CountryName'] == 'England'].copy()

In [ ]:
__version__ = "0.1.00"

import os # For file path handling  

# to obtain the pptx module:
#   pip install --user python-pptx
import pptx
from pptx.util import Mm
from pptx.util import Length


class PPT():
    OUTPUT_FILE_NAME = 'test_slides.pptx'
    IMAGE_FILE_NAME = 'test_chart.png'
    DIRECTORY_NAME = '.' # option to put in sub-directory
    
    # folder structure
    generated_chart = os.path.join(DIRECTORY_NAME, IMAGE_FILE_NAME)
    generated_ppt = os.path.join(DIRECTORY_NAME, OUTPUT_FILE_NAME)
    
    # slide template enumerations
    from enum import IntEnum
    class slide_template_layout(IntEnum):
        TITLE = 0
        TITLE_AND_CONTENT = 1
        TITLE_AND_MAX_CONTENT = 2
        
    placeholders = {
        slide_template_layout.TITLE:{'title':{'id':0, 'type':'text'}, 'subtitle':{'id':1, 'type':'text'}},
        slide_template_layout.TITLE_AND_CONTENT:{'title':{'id':0, 'type':'text'}, 'content':{'id':1, 'type':'content'}, 'footer':{'id':10, 'type':'text'}},
        slide_template_layout.TITLE_AND_MAX_CONTENT:{'title':{'id':0, 'type':'text'}, 'content':{'id':1, 'type':'content'}, 'footer':{'id':10, 'type':'text'}},
    }
    
    def __init__(self, template=None, outfile=None, placeholders=None, slide_width_mm=None, slide_height_mm=None):
        self.template = template
        self.outfile = outfile if outfile else self.generated_ppt
        self.placeholders = placeholders if placeholders else self.placeholders
        
        if template:
            print('Opening slides:', self.template)

        self.ppt = pptx.Presentation(self.template)
        self.slides = self.ppt.slides
        
        if (not slide_width_mm is None) and (not slide_height_mm is None):
            self.ppt.slide_width = Mm(slide_width_mm)
            self.ppt.slide_height = Mm(slide_height_mm)
        
    # Add slide
    def add_slide(self, layout):
        slide_layout = self.ppt.slide_layouts[layout]
        slide = self.slides.add_slide(slide_layout)
        return slide
    
    def set_text(self, slide, placeholder_name, text):
        try:
            layout = self.ppt.slide_layouts.index(slide.slide_layout)
            placeholder_id = self.placeholders.get(layout,{}).get(placeholder_name,{}).get('id',None)
            placeholder = slide.placeholders[placeholder_id]
            placeholder.text = text
        except Exception as e:
            print('Failed to add text to placeholder: ', placeholder_name)
            print(e)
            pass
        
    def add_textbox(self, slide, text, left_mm, top_mm, width_mm, height_mm):
        txtBox = slide.shapes.add_textbox(Mm(left_mm), Mm(top_mm), Mm(width_mm), Mm(height_mm))
        txtBox.text_frame.text = text
    
    def get_slide_placeholder_size(self, slide, placeholder_name):
        try:
            layout = self.ppt.slide_layouts.index(slide.slide_layout)
            placeholder_id = self.placeholders.get(layout,{}).get(placeholder_name,{}).get('id',None)
            placeholder = slide.placeholders[placeholder_id]
            sizes = (placeholder.left.mm, placeholder.top.mm, placeholder.width.mm, placeholder.height.mm)
        except Exception as e:
            print('Failed to get placeholder: ', placeholder_name)
            print(e)
            pass
        return sizes
    
    def get_slide_size(self):
        slide_width = self.ppt.slide_width
        slide_height = self.ppt.slide_height
        return (Length(slide_width).mm, Length(slide_height).mm)

    def add_image(self, slide, placeholder_name, image_file, keep_aspect_ratio=True, position='centre', delete_placeholder=True):
        try:
            # read image file to check actual aspect ratio
            img = plt.imread(image_file)
            img_height, img_width = img.shape[0], img.shape[1]
            image_aspect_ratio = img_width/img_height
            
            # get placeholder details, including aspect ratio
            layout = self.ppt.slide_layouts.index(slide.slide_layout)
            placeholder_id = self.placeholders.get(layout,{}).get(placeholder_name,{}).get('id',None)
            placeholder = slide.placeholders[placeholder_id]
            placeholder_aspect_ratio = placeholder.width/placeholder.height
            
            # default positions; later calculate offset for centering if appropriate
            x_pos = placeholder.left
            y_pos = placeholder.top
            width = placeholder.width
            height = placeholder.height
            
            # determine which dimensions to use
            if keep_aspect_ratio: # only set 1 dimension of image, to keep aspect ratio
                use_image_width=True
                if image_aspect_ratio < placeholder_aspect_ratio:
                    use_image_width=False

                if use_image_width:
                    if position=='centre':
                        new_img_height = img_height/img_width*width
                        y_pos = y_pos + (height - new_img_height)/2
                    pic = slide.shapes.add_picture(image_file, x_pos, y_pos, width=width)
                else:
                    if position=='centre':
                        new_img_width = img_width/img_height*height
                        x_pos = x_pos + (width - new_img_width)/2
                    pic = slide.shapes.add_picture(image_file, x_pos, y_pos, height=height)
                    
            else: # set both dimensions of image, which can therefore change aspect ratio
                pic = slide.shapes.add_picture(image_file, x_pos, y_pos, width=width, height=height)
                
            if delete_placeholder:
                sp = placeholder._sp # get shape xml element
                sp.getparent().remove(sp) # remove shape

        except Exception as e:
            print('Failed to add image to placeholder: ', placeholder_name, ', image filename:', image_file)
            print(e)
            pass

    def save(self, outfile=None):
        outfile = outfile if outfile else self.outfile
        if not outfile:
            print('Need to specify filename to save; existing')
            quit()
        self.ppt.save(outfile)

    def slide_describe(self, slide, layout_name):
        print('Slide layout:', layout_name)
        try:
            set_title(slide, layout_name)
        except:
            pass

        txBox = slide.shapes.add_textbox(0, 0, 100, 100)
        tf = txBox.text_frame
        tf.text = f'{layout_name}; Placeholders:'
        for placeholder in slide.placeholders:
            size_description = ("left=%d, top=%d, width=%d, height=%d (mm)" % (placeholder.left.mm, placeholder.top.mm, placeholder.width.mm, placeholder.height.mm))
            num_name = '%d; %s' % (placeholder.placeholder_format.idx, placeholder.name)
            name_name_size = num_name + " : " + size_description

            p = tf.add_paragraph()
            p.text = name_name_size

    def new_slide_describe(self, layout):
        slide = self.add_slide(layout)
        try:
            layout_name = self.ppt.slide_layouts[layout].name #layout.name
        except:
            layout_name = ''
        self.slide_describe(slide, layout_name)

    def add_describe_template(self, known_layouts_only = True):
        # to do: derive slide list to populate from template
        if not known_layouts_only:
            for slide_layout_num, slide_layout in enumerate(self.ppt.slide_layouts):
                self.new_slide_describe(slide_layout_num)
        else:
            self.new_slide_describe(self.slide_template_layout.TITLE)
            self.new_slide_describe(self.slide_template_layout.TITLE_AND_CONTENT)
            self.new_slide_describe(self.slide_template_layout.TITLE_AND_MAX_CONTENT)

In [ ]:
annotate_template = False
if annotate_template:
    print('Describe slide template')
    prs = PPT(template_pptx) # create new presentation using default template
    prs.add_describe_template(known_layouts_only=False)
    prs.save('template_annotated.pptx')
    
if export_slides:
    # Create new presentation from (empty) template
    prs = PPT(template_pptx) # create new presentation using default template

In [ ]:
slide_title = 'COVID-19 Summary'
display_h1(slide_title)
text_str = 'Analysis run: '+ run_date_str + '\n'
text_str = text_str + 'Latest data: '+ latest_date_str + '\n'
text_str = text_str + 'Total confirmed cases: {:,d}'.format(df_totals['confirmed'].sum().astype(int)) + '\n'
text_str = text_str + 'Total deaths: {:,d}'.format(df_totals['deaths'].sum().astype(int)) + '\n'
text_str = text_str + '\n'
text_str = text_str + 'COVID-19 data: COVID Data Repository by Johns Hopkins CSSE' + '\n'
text_str = text_str + 'Mortality data: Short-term Mortality Fluctuations at mortality.org' + '\n'
text_str = text_str + 'UK data from: https://coronavirus.data.gov.uk' +'\n'
text_str = text_str + '\n'

text_str = text_str + 'Analysis & summary by Colin Osborne' + '\n'
text_str = text_str + 'Country/region included in trends if confirmed cases above: {:,d}'.format(num_cases_threshold)

print(text_str)

if export_slides:
    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.set_text(slide, placeholder_name='content', text=text_str)

In [ ]:
slide_title = 'Covid-19 Worldwide'
display_h1(slide_title)
small_subset = True
measure_daily = True
independent_y_scales = True

chart_width_large = 350
chart_width_small = 200
chart_height = 100

def trend_chart_world(df, scale_type='linear', x_metric='date', metric='deaths', x_min=None, x_max=None, measure_daily=True, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()

    if measure_daily:
        df[metric] = df[metric].diff().clip(lower=0)
    x_metric_type = ':T' # for date
    if x_metric == 'days_since_100':
        df = df[df['days_since_100']>=0]
        x_metric_type = ':Q' # for count of days
        x_min = 0 # no need for negative x-axis
        
    chart = alt.Chart().mark_area().encode(
        x   = 'date:T',
        y   = metric+':Q',
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip(metric+':Q', format=',d')]
    ).properties(height=chart_height, width=chart_width_large)

    line = alt.Chart().mark_line(
        color='red',
    ).transform_window(
        weekAvg=f'mean({metric})',
        frame=[-7, 0],
        groupby=['country']
    ).encode(
        x='date:T',
        y=alt.Y('weekAvg:Q', title=row_title),
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    )

    chart = alt.layer(chart, line, data=df)
        
    chart = chart.properties(title=column_title)

    return chart

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_world_totals(df, category='', metric='confirmed', row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()
    df['metric'] = df[metric] # didn't find a way to successfully use 'metric' variable inside transform_aggregate
    df['scope'] = 'World'

    bars = alt.Chart(df).mark_bar(
    ).transform_aggregate(
        total = 'sum(metric)',
        groupby = ['scope']
    ).encode(
        x='total:Q',
        y=alt.Y('scope:N', title=row_title),
        tooltip=[alt.Tooltip('scope:N'),
                 alt.Tooltip('total:Q', format=',d')]
    )

    text = bars.mark_text(
        align = 'right',
        baseline = 'middle',
        dx = -15,
        color = 'white',
        fontSize=20
    ).encode(
        text=alt.Text('total:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width_small)
    
    return chart

def trend_chart_continents(df, scale_type='linear', x_metric='date', metric='deaths', x_min=None, x_max=None, measure_daily=True, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()

    df = df.groupby(['continent', 'date']).agg({'confirmed':'sum', 'deaths':'sum', 'Population':'sum'})
    df = df.reset_index()

    df['deaths_per_M_population']    = df['deaths']    / df['Population'] * 1E6 # per 1M population
    df['confirmed_per_M_population'] = df['confirmed'] / df['Population'] * 1E6 # per 1M population

    if measure_daily:
        df = df.loc[df[metric]!=0] # smooth any irregular reporting
        df[metric] = df[metric].transform(lambda x: x.diff()).clip(lower=0)
    x_metric_type = ':T' # for date
    if x_metric == 'days_since_100':
        df = df[df['days_since_100']>=0]
        x_metric_type = ':Q' # for count of days
        x_min = 0 # no need for negative x-axis
        
    y_max = df[metric].max()
    y_min = df[metric].min()
    if scale_type == 'log':
        y_min = max(y_min, 0.1)
        small_non_zero_value = 0.001
        df.loc[df['weekAvg']==0, 'weekAvg'] = small_non_zero_value # to enable log calculation without errors

    chart = alt.Chart(data=df).mark_line(
    ).transform_window(
        weekAvg=f'mean({metric})',
        frame=[-7, 0],
        groupby=['continent']
    ).encode(
        x='date:T',
        y=alt.Y('weekAvg:Q', title=row_title),
        color=alt.Color('continent:N', sort=None),
        tooltip=[alt.Tooltip('continent:N'),
                 alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    ).properties(height=chart_height, width=chart_width_large)

    chart = chart.properties(title=column_title)

    return chart

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_coloured_world(df, category='continent', sort_field='deaths', data_field='confirmed', row_title='', column_title=''):
    df = df.copy()

    sort_order = None # initial value
    if sort_field and category:
        sort_order = list(df.sort_values(by=sort_field, ascending=False)[category].unique())

    bars = alt.Chart(df).mark_bar().encode(
        x=alt.X(data_field+':Q', title=data_field),
        y=alt.Y(f'{category}:N', sort=sort_order, title=row_title), # sort='-x' to sort locally in this chart, or 'None' for no sort
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(data_field+':Q', format=',d')]
    ).properties(
        height=chart_height, width=chart_width_small,
#         title='Covid total '+data_field+', '+continent
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(data_field+':Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title)

    return chart

def configure_trend_chart_world(chart):
#     chart = chart.configure_headerRow(
# #         title=None,
# #         labelColor='blue',
# #         labelFontSize=20
#     )

    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    return chart

def df_world_time_series_summary(df):
    df_totals = df.copy().drop(['continent', 'Combined_Key', 'country', 'state', 'critical_estimate', 'days_since_100', 'type'], axis=1)
    df_totals = df_totals.groupby(['date']).agg({'lat':'mean', 'long':'mean', 'confirmed':'sum', 'deaths':'sum', 'Population':'sum'})
    df_totals = df_totals.reset_index()

    return df_totals

def df_continents_time_series_summary(df):
    df_totals = df.copy().drop(['Combined_Key', 'country', 'state', 'critical_estimate', 'days_since_100', 'type'], axis=1)
    df_totals = df_totals.groupby(['date', 'continent']).agg({'lat':'mean', 'long':'mean', 'confirmed':'sum', 'deaths':'sum', 'Population':'sum'})
    df_totals = df_totals.reset_index()

    return df_totals

def world_chart_summary(df):
    df_totals = get_totals_from_time_series(df)

    df_continent_totals_absolute       = get_continent_totals(df_totals)

    df_world_time_series = df_world_time_series_summary(df)
    df_continents_time_series = df_continents_time_series_summary(df)

    trend_confirmed_cases  = trend_chart_world(df_world_time_series, metric='confirmed', row_title='World', column_title='Daily Confirmed Cases')
    trend_deaths           = trend_chart_world(df_world_time_series, metric='deaths', column_title='Daily Deaths')
    world_total_confirmed  = get_bar_chart_world_totals(df_continent_totals_absolute, metric='confirmed', row_title='', column_title='Total Confirmed Cases')
    world_total_deaths     = get_bar_chart_world_totals(df_continent_totals_absolute, metric='deaths', row_title='', column_title='Total Deaths')
    world_row              = trend_confirmed_cases | trend_deaths | world_total_confirmed | world_total_deaths
    
    trend_confirmed_cases  = trend_chart_continents(df, 'linear', metric='confirmed', row_title='Continents', column_title='Daily Confirmed Cases')
    trend_deaths           = trend_chart_continents(df, 'linear', metric='deaths', row_title='', column_title='Daily Deaths')
    total_confirmed_cases  = get_bar_chart_coloured_world(df_continent_totals_absolute, category='continent', data_field='confirmed', sort_field='deaths', row_title='', column_title='Total Confirmed Cases')
    total_deaths           = get_bar_chart_coloured_world(df_continent_totals_absolute, category='continent', data_field='deaths', sort_field='deaths', row_title='', column_title='Total Deaths')
    continent_row          = trend_confirmed_cases | trend_deaths | total_confirmed_cases | total_deaths
    
    trend_confirmed_norm   = trend_chart_continents(df, 'linear', metric='confirmed_per_M_population', row_title='Continents', column_title='Daily Confirmed Cases per M population')
    trend_deaths_norm      = trend_chart_continents(df, 'linear', metric='deaths_per_M_population', row_title='', column_title='Daily Deaths per M population')
    total_confirmed_norm   = get_bar_chart_coloured_world(df_continent_totals_absolute, category='continent', data_field='confirmed_per_M_population', sort_field='deaths_per_M_population', row_title='', column_title='Total Confirmed Cases per M population')
    total_deaths_norm      = get_bar_chart_coloured_world(df_continent_totals_absolute, category='continent', data_field='deaths_per_M_population', sort_field='deaths_per_M_population', row_title='', column_title='Total Deaths per M population')
    continent_norm_row     = trend_confirmed_norm | trend_deaths_norm | total_confirmed_norm | total_deaths_norm
    
    chart = world_row & continent_row & continent_norm_row

    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart_world(chart)
    
    return chart
    
footer = 'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

chart = world_chart_summary(df_covid_time_series)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
display_h1('Trend Charts by Continent')
small_subset = False
top_n = 10

measure_daily = True
include_total_confirmed_bar_chart = True

sort_field_absolute = 'confirmed' if sort_by_confirmed else 'deaths'
sort_field_per_population = 'confirmed_per_M_population' if sort_by_confirmed else 'deaths_per_M_population'

# if small_subset:
#     Combined_Key_list = ['Italy', 'Sweden', 'United Kingdom', 'France', 'US', 'Germany', 'Brazil']
#     print('Country list:', Combined_Key_list)
#     df_covid_time_series_filtered = df_covid_time_series[df_covid_time_series['Combined_Key'].isin(Combined_Key_list)].copy()
# else:
#     df_covid_time_series_filtered = df_covid_time_series.copy()

print(f'Per continent, top {top_n} by total {sort_field_absolute}; '+latest_date_str)


def trend_chart_continent(df, continent, scale_type='linear', x_metric='date', metric='deaths', x_min=None, x_max=None, measure_daily=True, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()
    
    if measure_daily:
        df[metric] = df[metric].diff().clip(lower=0)
    x_metric_type = ':T' # for date
    if x_metric == 'days_since_100':
        df = df[df['days_since_100']>=0]
        x_metric_type = ':Q' # for count of days
        x_min = 0 # no need for negative x-axis
        
    df['weekAvg'] = df.groupby('Combined_Key')[metric].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)

    y_max = df['weekAvg'].max()
    y_min = df['weekAvg'].min()
    if scale_type == 'log':
        y_min = max(y_min, 0.1)
        small_non_zero_value = 0.001
        df.loc[df['weekAvg']==0, 'weekAvg'] = small_non_zero_value # to enable log calculation without errors

    line = alt.Chart(df).mark_line(
    ).encode(
        x=alt.X(x_metric+x_metric_type, scale=alt.Scale(domain=[x_min, x_max])),
        y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(type=scale_type, domain=[y_min, y_max], clamp=True)),
        color=alt.Color('Combined_Key:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('Combined_Key:N'),
                 alt.Tooltip(x_metric+x_metric_type),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    ).properties(
        height=110, width=450,
#         title='Covid daily '+metric+', '+continent
    )
    
    chart = line
    chart = chart.properties(title=column_title)

    return chart

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_coloured(df, continent, category='continent', data_field='confirmed', row_title='', column_title=''):
    bars = alt.Chart(df).mark_bar().encode(
        x=alt.X(data_field+':Q', title=data_field),
        y=alt.Y(category+':N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
        color=alt.Color('Combined_Key:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(category+':N'),
                 alt.Tooltip(data_field+':Q', format=',d')]
    ).properties(
        height=110, width=200,
#         title='Covid total '+data_field+', '+continent
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(data_field+':Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title)

    return chart

def configure_trend_chart_continents(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    chart = chart.configure_axisX(
#         title=None
    )

    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    return chart

def trend_dashboard_charts(df, df_totals, sort_field, confirmed_field, deaths_field, scale_type='linear', x_metric='date'):
    chart = alt.vconcat()
    first_row = True
    x_min = df[x_metric].min()
    x_max = df[x_metric].max()
    
    for continent in df['continent'].unique():
        df_filtered = df[df['continent']==continent]
        if not top_n is None:
            df_totals_filtered = df_totals.copy()
            df_totals_filtered = df_totals_filtered[df_totals_filtered['continent']==continent]
            df_totals_filtered = df_totals_filtered.sort_values(by=sort_field, ascending=False)
            df_totals_filtered = df_totals_filtered[:top_n]
            combined_key_top_n = df_totals_filtered['Combined_Key']
            df_filtered = df_filtered[df_filtered['Combined_Key'].isin(combined_key_top_n)] # would like to put it in the order of highest totals

        trend_confirmed_cases = trend_chart_continent(df_filtered, continent, scale_type, x_metric=x_metric, metric=confirmed_field, x_min=x_min, x_max=x_max,
                                                      measure_daily=measure_daily, row_title=continent, column_title='Daily Confirmed Cases' if first_row else '')
        trend_deaths          = trend_chart_continent(df_filtered, continent, scale_type, x_metric=x_metric, metric=deaths_field, x_min=x_min, x_max=x_max,
                                                      measure_daily=measure_daily, row_title='', column_title='Daily Deaths' if first_row else '')

        chart_confirmed = get_bar_chart_coloured(df_totals_filtered, continent, category='Combined_Key', data_field=confirmed_field, column_title='Total Confirmed Cases' if first_row else '')
        chart_deaths    = get_bar_chart_coloured(df_totals_filtered, continent, category='Combined_Key', data_field=deaths_field, column_title='Total Deaths' if first_row else '')
        first_row = False

        row = (trend_confirmed_cases | trend_deaths)
        if include_total_confirmed_bar_chart:
            row = row  | chart_confirmed
        row = row | chart_deaths
        chart = (chart & row).resolve_scale(color='independent').configure_legend(title=None, labelFontSize=11, symbolSize=15)
    
    return chart

slide_title = 'COVID-19 Trend - Absolute'
footer = f'Per continent, top {top_n} by total {sort_field_absolute}; trend charts using 7 day rolling mean'
display_h2(slide_title)

chart = trend_dashboard_charts(df_covid_time_series, df_totals, sort_field=sort_field_absolute, confirmed_field='confirmed', deaths_field='deaths')
chart = configure_trend_chart_continents(chart)

chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)
    

In [ ]:
slide_title = 'Covid-19 UK'
display_h1(slide_title)

chart_width_large = 350
chart_width_small = 200
chart_height = 150
# chart_height_large = 350

def trend_chart_uk_overall(df, measure_daily=True, category='Area name', area_type='Nation', scale_type='linear', metric='deaths', x_min=None, x_max=None, y_min=None, y_max=None, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()

    df = df[df['Area type']==area_type]
    df = df.groupby(['date']).agg({'confirmed':'sum', 'deaths':'sum', 'confirmedNorm':'sum', 'deathsNorm':'sum'})
    df = df.reset_index()

    df[metric] = df[metric].transform(lambda x: x.diff()).clip(lower=0)
        
    if y_max is None: y_max = df[metric].max()
    if y_min is None: y_min = df[metric].min()

    chart = alt.Chart().mark_area().encode(
        x   = 'date:T',
        y   = metric+':Q',
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip(metric+':Q', format=',d')]
    ).properties(height=chart_height, width=chart_width_large)

    line = alt.Chart().mark_line(
        color='red',
    ).transform_window(
        weekAvg=f'mean({metric})',
        frame=[-7, 0],
        groupby=['Area name']
    ).encode(
        x='date:T',
        y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], clamp=True)),
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    )

    chart = alt.layer(chart, line, data=df)
    chart = chart.properties(title=column_title)

    return chart

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_latest_daily_uk(df, category='', metric='confirmed', row_title='', column_title=''):
    df = df.copy()

    df[metric] = df[metric].transform(lambda x: x.diff()).clip(lower=0)
    if category:
        df = df.groupby(['Area type']).last()
    else:
        df['scope'] = 'UK'
        category = 'scope'
        df = df[df['date']==df['date'].max()] # last value
    
    df = df.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    bars = alt.Chart().mark_bar(
    ).encode(
        x=f'{metric}:Q',
        y=alt.Y(f'{category}:N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    text = bars.mark_text(
        align = 'right',
        baseline = 'middle',
        dx = -15,
        fontSize = 20,
#         color='black' # not working for some reason
    ).encode(
        color=alt.Color(f'{category}', legend=None, scale=alt.Scale(range=['white'])), # sets the whole bar colour for some reason
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = alt.layer(bars, text, data=df)
    chart = chart.resolve_scale(color='independent')
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width_small)
    
    return chart

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_uk_overall(df, category='', area_type='Nation', metric='confirmed', row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()

    df = df[df['Area type']==area_type]
    df = df.groupby(['date']).agg({'confirmed':'sum', 'deaths':'sum', 'confirmedNorm':'sum', 'deathsNorm':'sum'})
    df = df.reset_index()

    df[metric] = df[metric].transform(lambda x: x.diff()).clip(lower=0)
    
    if category:
        df = df.groupby([category]).agg({'confirmed':'sum', 'deaths':'sum'})
    else:
        df['scope'] = 'UK'
        category = 'scope'
        df = df.groupby([category]).agg({'confirmed':'sum', 'deaths':'sum'})
    df = df.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    bars = alt.Chart(df).mark_bar(
    ).encode(
        x=f'{metric}:Q',
        y=alt.Y(f'{category}:N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    text = bars.mark_text(
        align = 'right',
        baseline = 'middle',
        dx = -15,
        color = 'white',
        fontSize=20
    ).encode(
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width_small)
    
    return chart

def trend_chart_uk_areas(df, scale_type='linear', category='RegionName', area_type='Region', metric='deaths', x_min=None, x_max=None, y_min=None, y_max=None, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()
    
    x_min = df['date'].min() if not x_min else x_min
    x_max = df['date'].max() if not x_max else x_max

    df = df[df['Area type']==area_type]
    df = df.groupby([category, 'date']).agg({'confirmed':'sum', 'deaths':'sum', 'confirmedNorm':'sum', 'deathsNorm':'sum'})
    df = df.loc[df[metric]!=0] # smooth any irregular reporting
    df = df.reset_index()

    df[metric] = df[metric].transform(lambda x: x.diff()).clip(lower=0)
        
    if y_max is None: y_max = df[metric].max()
    if y_min is None: y_min = df[metric].min()
        
    if scale_type == 'log':
        y_min = max(y_min, 0.1)
        small_non_zero_value = 0.001
        df.loc[df['weekAvg']==0, 'weekAvg'] = small_non_zero_value # to enable log calculation without errors

    chart = alt.Chart(data=df).mark_line(
    ).transform_window(
        weekAvg=f'mean({metric})',
        frame=[-7, 0],
        groupby=[category] # 'Area name'
    ).encode(
#         x='date:T',
        x=alt.X('date:T', scale=alt.Scale(domain=[x_min, x_max])),
        y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], clamp=True)),
        color=alt.Color(category+':N', sort=None),
        tooltip=[alt.Tooltip(category+':N'),
                 alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    ).properties(height=chart_height, width=chart_width_large)

    chart = chart.properties(title=column_title)

    return chart

# data_field: 'confirmed' or 'deaths'
def get_bar_chart_uk_areas(df, sort_field='deaths', category='RegionName', area_type='Region', rollup='latest', measure_daily=True, metric='confirmed', row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()
    
    df = df[df['Area type']==area_type]
    df = df.groupby(['date', category]).agg({'confirmed':'sum', 'deaths':'sum', 'confirmedNorm':'sum', 'deathsNorm':'sum'})
    df = df.loc[df[metric]!=0] # smooth any irregular reporting
    df = df.reset_index()
    
    if measure_daily:
        df = df.set_index(['Area name'], append=True)
        df[metric] = df.groupby(['Area name'])[metric].transform(lambda x: x.diff()).clip(lower=0)
        df = df.reset_index()
    x_metric_type = ':T' # for date

    if rollup=='latest':
        df = df[df['date']==df['date'].max()] # last value
    else: # total
        df = df.groupby([category]).agg({'confirmed':'sum', 'deaths':'sum', 'confirmedNorm':'sum', 'deathsNorm':'sum'})
        df = df.reset_index()
        
    sort_order = None # initial value
    if sort_field:
        sort_order = list(df.sort_values(by=sort_field, ascending=False)[category].unique())

    df = df.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    bars = alt.Chart(df).mark_bar(
    ).encode(
        x=f'{metric}:Q',
        y=alt.Y(f'{category}:N', sort=sort_order, title=row_title), # sort='-x' to sort locally in this chart, or 'None' for no sort
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(category+':N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width_small)
    
    return chart

def configure_trend_chart_uk(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    return chart

def df_area_top_n(df, area_type='Region', sort_field='deaths', top_n=None):
    df = df.copy()    
    if not top_n is None:
        df = df[df['Area type']=='Upper tier local authority']
        df_totals = df[df['date']==df['date'].max()] # last value
        df_totals = df_totals.sort_values(by=sort_field, ascending=False)
        df_totals = df_totals[:top_n]
        authority_top_n = df_totals['Area name']
        df = df[df['Area name'].isin(authority_top_n)] # would like to put it in the order of highest totals
    return df
    
def uk_chart_summary(df):
    df = df.copy()
    
    x_min = df['date'].min()
    x_max = df['date'].max()

    trend_confirmed_cases  = trend_chart_uk_overall(df, area_type='Nation', metric='confirmed', x_min=x_min, x_max=x_max, y_min=0, y_max=5000, row_title='UK', column_title='Daily Confirmed Cases')
    trend_deaths           = trend_chart_uk_overall(df, area_type='Nation', metric='deaths', x_min=x_min, x_max=x_max, column_title='Daily Deaths')
    total_confirmed_cases  = get_bar_chart_uk_overall(df, area_type='Nation', metric='confirmed', row_title='', column_title='Total Confirmed Cases')
    total_confirmed_deaths = get_bar_chart_uk_overall(df, area_type='Nation', metric='deaths', row_title='', column_title='Total Deaths')
    uk_trend_row           = trend_confirmed_cases | trend_deaths | total_confirmed_cases | total_confirmed_deaths
    
    trend_confirmed_cases  = trend_chart_uk_areas(df, category='Area name', area_type='Nation', metric='confirmed', x_min=x_min, x_max=x_max, y_min=0, y_max=5000, row_title='Nations', column_title='Daily Confirmed Cases')
    trend_deaths           = trend_chart_uk_areas(df, category='Area name', area_type='Nation', metric='deaths', x_min=x_min, x_max=x_max, row_title='', column_title='Daily Deaths')
    total_confirmed_cases  = get_bar_chart_uk_areas(df, category='Area name', area_type='Nation', rollup='total', metric='confirmed', sort_field='deaths', row_title='', column_title='Total Confirmed Cases')
    total_deaths           = get_bar_chart_uk_areas(df, category='Area name', area_type='Nation', rollup='total', metric='deaths', sort_field='deaths', row_title='', column_title='Total Deaths')
    uk_nations_trend_row   = trend_confirmed_cases | trend_deaths | total_confirmed_cases | total_deaths
    
    trend_confirmed_norm   = trend_chart_uk_areas(df, category='Area name', area_type='Nation', metric='confirmedNorm', x_min=x_min, x_max=x_max, y_min=0, y_max=20, row_title='Nations', column_title='Daily Confirmed Cases per 100k')
    trend_deaths_norm      = trend_chart_uk_areas(df, category='Area name', area_type='Nation', metric='deathsNorm', x_min=x_min, x_max=x_max, row_title='', column_title='Daily Deaths per 100k')
    total_confirmed_norm   = get_bar_chart_uk_areas(df, category='Area name', area_type='Nation', rollup='total', metric='confirmedNorm', sort_field='deathsNorm', row_title='', column_title='Total Confirmed Cases per 100k')
    total_deaths_norm      = get_bar_chart_uk_areas(df, category='Area name', area_type='Nation', rollup='total', metric='deathsNorm', sort_field='deathsNorm', row_title='', column_title='Total Deaths per 100k')
    uk_nations_norm_row    = trend_confirmed_norm | trend_deaths_norm | total_confirmed_norm | total_deaths_norm
    
    chart = uk_trend_row & uk_nations_trend_row & uk_nations_norm_row

    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart_uk(chart)
    
    return chart
    
footer = 'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

print('England: Data include only pillar 1 cases until 2 July, from when pillar 2 cases are also included.')
chart = uk_chart_summary(df_time_series_uk)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Covid-19 UK Regions'
display_h1(slide_title)

chart_width_large = 350
chart_width_small = 200
chart_height = 200
# chart_height_large = 350
 
def uk_chart_summary(df):
    df = df.copy()
    
    x_min = df['date'].min()
    x_max = df['date'].max()
    
    trend_confirmed_cases  = trend_chart_uk_areas(df, category='Area name', area_type='Region', metric='confirmed', x_min=x_min, x_max=x_max, row_title='Regions', column_title='Daily Confirmed Cases')
    trend_deaths           = trend_chart_uk_areas(df, category='Area name', area_type='Region', metric='deaths', x_min=x_min, x_max=x_max, row_title='', column_title='Daily Deaths')
    total_confirmed_cases  = get_bar_chart_uk_areas(df, category='Area name', area_type='Region', rollup='total', metric='confirmed', sort_field='confirmed', row_title='', column_title='Total Confirmed Cases')
    total_deaths           = get_bar_chart_uk_areas(df, category='Area name', area_type='Region', rollup='total', metric='deaths', sort_field='deaths', row_title='', column_title='Total Deaths')
    uk_regions_row         = trend_confirmed_cases | trend_deaths | total_confirmed_cases | total_deaths
    
    trend_confirmed_norm   = trend_chart_uk_areas(df, category='Area name', area_type='Region', metric='confirmedNorm', row_title='Regions', column_title='Daily Confirmed Cases per 100k')
    trend_deaths_norm      = trend_chart_uk_areas(df, category='Area name', area_type='Region', metric='deathsNorm', x_min=x_min, x_max=x_max, row_title='', column_title='Daily Deaths per 100k')
    total_confirmed_norm   = get_bar_chart_uk_areas(df, category='Area name', area_type='Region', rollup='total', metric='confirmedNorm', sort_field='confirmedNorm', row_title='', column_title='Total Confirmed Cases per 100k')
    total_deaths_norm      = get_bar_chart_uk_areas(df, category='Area name', area_type='Region', rollup='total', metric='deathsNorm', sort_field='deathsNorm', row_title='', column_title='Total Deaths per 100k')
    uk_regions_norm_row    = trend_confirmed_norm | trend_deaths_norm | total_confirmed_norm | total_deaths_norm

    chart = uk_regions_row & uk_regions_norm_row

    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart_uk(chart)
    
    return chart
    
footer = 'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

print('Some of the data is England only, not full UK. Will try to include other nations of UK in future.')
print('England: Data include only pillar 1 cases until 2 July, from when pillar 2 cases are also included.')
chart = uk_chart_summary(df_time_series_uk)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Upper Tier Authorities'
display_h1(slide_title)
top_n = 15

chart_height = 250
trend_chart_width = 350
totals_chart_width = 200

def trend_chart_region_facet_item(df, sorted_order=None, sort_field='deaths', scale_type='linear', x_metric='date', metric='deaths', x_min=None, x_max=None, y_min=None, y_max=None, measure_daily=True, row_title='', column_title='', chart_height=250, chart_width=350):
    df = df.copy()
    df = df.reset_index()
    
    df = df.dropna(axis='index', subset=[metric])

    if measure_daily:
        df = df.set_index(['Area name'], append=True)
        df[metric] = df.groupby(['Area name'])[metric].transform(lambda x: x.diff()).clip(lower=0)
        df = df.reset_index()
#         df[metric] = df[metric].transform(lambda x: x.diff()).clip(lower=0)
        
    x_metric_type = ':T' # for date
    if x_metric == 'days_since_100':
        df = df[df['days_since_100']>=0]
        x_metric_type = ':Q' # for count of days
        x_min = 0 # no need for negative x-axis
        
    df['weekAvg'] = df.groupby(['Area name'])[metric].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)

    if y_max is None: y_max = df['weekAvg'].max()
    if y_min is None: y_min = df['weekAvg'].min()
    if scale_type == 'log':
        y_min = max(y_min, 0.1)
        small_non_zero_value = 0.001
        df.loc[df['weekAvg']==0, 'weekAvg'] = small_non_zero_value # to enable log calculation without errors

    line = alt.Chart(df).mark_line(
    ).encode(
        x=alt.X(x_metric+x_metric_type, scale=alt.Scale(domain=[x_min, x_max])),
        y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(type=scale_type, domain=[y_min, y_max], clamp=True)),
        color=alt.Color('Area name:N', sort=sorted_order, legend=None, scale=alt.Scale(scheme='category20')),
        tooltip=[alt.Tooltip('Area name:N'),
                 alt.Tooltip(x_metric+x_metric_type),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    ).properties(
        height=chart_height, width=chart_width,
#         title='Covid daily '+metric+', '+continent
    )
    
#     chart = alt.layer(line, data=df).facet(alt.Column(f'{facet_field}:N', title=None), columns=3)#.resolve_scale(y='independent')
    chart = line
    chart = chart.properties(title=column_title)

    return chart

# data_field: 'confirmed' or 'deaths'
def get_bar_chart_uk_areas_facet_item(df, sorted_order=None, sort_field='deaths', category='RegionName', rollup='latest', measure_daily=True, metric='confirmed', x_min=None, x_max=None, row_title='', column_title='', chart_height=200, chart_width=350):
    df = df.copy()
    df = df.reset_index()
    
    if measure_daily:
        df = df.set_index(['Area name'], append=True)
        df[metric] = df.groupby(['Area name'])[metric].transform(lambda x: x.diff()).clip(lower=0)
        df = df.reset_index()
    x_metric_type = ':T' # for date

    if rollup=='latest':
        df = df[df['date']==df['date'].max()] # last value
    else: # total
        df = df.groupby([category]).agg({'confirmed':'sum', 'deaths':'sum', 'confirmedNorm':'sum'})
        df = df.reset_index()
        
    sort_order = None # initial value
    if sort_field:
        sort_order = list(df.sort_values(by=sort_field, ascending=False)[category].unique())

    df = df.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    if x_max is None: x_max = df[metric].max()
    if x_min is None: x_min = df[metric].min()

    bars = alt.Chart(df).mark_bar(
    ).encode(
        x=alt.X(f'{metric}:Q', scale=alt.Scale(domain=[x_min, x_max])),
        y=alt.Y(f'{category}:N', sort=sort_order, title=row_title), # sort='-x' to sort locally in this chart, or 'None' for no sort
        color=alt.Color(f'{category}:N', sort=sorted_order, legend=None, scale=alt.Scale(scheme='category20')),
        tooltip=[alt.Tooltip(category+':N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = alt.layer(bars, text).properties(height=chart_height, width=chart_width)
    chart = chart.properties(title=column_title)
    
    return chart

def facet_wrap(subplts, plots_per_row):
    rows = [subplts[i:i+plots_per_row] for i in range(0, len(subplts), plots_per_row)]
    compound_chart = alt.vconcat()
    for r in rows:
        rowplot = alt.vconcat() #start a new row
        for item in r:
            rowplot |= item #add suplot to current row as a new column
        compound_chart &= rowplot # add the entire row of plots as a new row

    return compound_chart

def df_get_daily_average_and_totals(df, index_col_list, metric):
    df_daily = df.copy()
    df_daily = df_daily.dropna(axis='index', subset=[metric])
    df_daily = df_daily.set_index(index_col_list, append=True)
    df_daily[metric] = df_daily.groupby(index_col_list)[metric].transform(lambda x: x.diff()).clip(lower=0)
    df_totals = pd.DataFrame()
    if not df_daily.empty:
        df_totals = df_daily.groupby(index_col_list).agg({metric:'sum'})
        df_daily['weekAvg'] = df_daily.groupby(index_col_list)[metric].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
    return df_daily.reset_index(), df_totals.reset_index()
    
def configure_facet_wrap_charts(chart):
    chart = chart.configure_headerRow(
        title=None,
    )
    
    chart = chart.configure_header(
        labelFontSize=16
    )

    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    
    return chart

def uk_regions_trend_dashboard_charts(df, filter_area_type='Upper tier local authority', filter_area_name_list=None, normalised=False, multiple_facets=True, plots_per_row=3, scale_type='linear', x_metric='date', top_n=None, chart_height=250, trend_chart_width=350, totals_chart_width=200):
    df = df.copy()
    
    if not normalised:
        confirmed_field = 'confirmed'
        deaths_field = 'deaths'
        normalised_title_string = ''
    else:
        confirmed_field = 'confirmedNorm'
        deaths_field = 'deathsNorm' # not in data yet
        normalised_title_string = ' per 100k Population'
    sort_field = confirmed_field
    
    chart = alt.vconcat()
    first_row = True
    if filter_area_type:
        df = df[df['Area type'] == filter_area_type]
    if filter_area_name_list:
        df = df[df['Area name'].isin(filter_area_name_list)]
    
    x_min = df[x_metric].min()
    x_max = df[x_metric].max()
    
    if not multiple_facets: # simple approach for now
        df['RegionName'] = 'Combined'
        
    authority_list_top_n = []
    if top_n is None:
        df_filtered = df.copy()
    else:
        for region in df['RegionName'].unique():
            df_filtered = df[df['RegionName']==region]
            if (not df_filtered.empty) and (sort_field in df_filtered):
                df_filtered_daily, df_filtered_totals = df_get_daily_average_and_totals(df_filtered, index_col_list=['Area name'], metric=confirmed_field)
                if not df_filtered_daily.empty:
                    df_filtered_totals = df_filtered_totals.sort_values(by=sort_field, ascending=False)
                    df_filtered_totals = df_filtered_totals[:top_n]
                    authority_top_n = list(df_filtered_totals['Area name'].unique())
        #             print('Region:', region, '; authority_list_top_n:', authority_top_n)
                    authority_list_top_n = authority_list_top_n + authority_top_n
        df_filtered = df[df['Area name'].isin(authority_list_top_n)] # would like to put it in the order of highest totals

    chart = alt.Chart()
    df_daily, df_totals = df_get_daily_average_and_totals(df_filtered, index_col_list=['Area name'], metric=confirmed_field)
    if not df_daily.empty:
        daily_avg_max = df_daily['weekAvg'].max()
        total_max = df_totals[confirmed_field].max()
    
        trend_chart_list = []
        totals_chart_list = []
        for region in df['RegionName'].unique():
            df_facet = df_filtered[df_filtered['RegionName']==region]
            sorted_order = df_facet.sort_values(by=sort_field, ascending=False)['Area name'].unique()
            trend_chart = trend_chart_region_facet_item(df_facet, sorted_order=sorted_order, sort_field=confirmed_field, scale_type=scale_type, x_metric=x_metric, metric=confirmed_field, x_min=x_min, x_max=x_max, y_min=0, y_max=daily_avg_max,
                                                        measure_daily=True, row_title='', column_title=region, chart_height=chart_height, chart_width=trend_chart_width)
            totals_chart = get_bar_chart_uk_areas_facet_item(df_facet, sorted_order=sorted_order, sort_field=confirmed_field, category='Area name', rollup='total', measure_daily=True, metric=confirmed_field, x_min=0, x_max=total_max,
                                                             row_title='', column_title=region, chart_height=chart_height, chart_width=totals_chart_width)
            trend_chart_list.append(trend_chart)
            totals_chart_list.append(totals_chart)

        trend_facet_chart = facet_wrap(trend_chart_list, plots_per_row=plots_per_row).properties(title='Daily Confirmed Cases'+normalised_title_string)
        totals_facet_chart = facet_wrap(totals_chart_list, plots_per_row=plots_per_row).properties(title='Total Confirmed Cases'+normalised_title_string)

        chart = trend_facet_chart | totals_facet_chart

    return chart

if not include_uk_local_authorities:
    print('Not configured to show UK local authorities')
else:
    footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} authorities by total confirmed cases'

    chart = uk_regions_trend_dashboard_charts(df_time_series_england, filter_area_type='Upper tier local authority', normalised=False, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Selected Areas'
display_h1(slide_title)

if not include_uk_local_authorities:
    print('Not configured to show UK local authorities')
else:
    chart_height = 300
    trend_chart_width = 1000
    totals_chart_width = 600

    footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

    filter_area_name_list=['New Forest', 'South Cambridgeshire', 'Cambridge', 'Southampton', 'Huntingdonshire', 'East Suffolk', 'Leicester']
    chart_abs = uk_regions_trend_dashboard_charts(df_time_series_uk, filter_area_type='Lower tier local authority', filter_area_name_list=filter_area_name_list, normalised=False, multiple_facets=False, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart_norm = uk_regions_trend_dashboard_charts(df_time_series_uk, filter_area_type='Lower tier local authority', filter_area_name_list=filter_area_name_list, normalised=True, multiple_facets=False, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart = chart_abs & chart_norm
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Upper Tier Authorities per 100k Population'
display_h1(slide_title)

if not include_uk_local_authorities:
    print('Not configured to show UK local authorities')
else:
    chart_height = 250
    trend_chart_width = 350
    totals_chart_width = 200

    footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} authorities by total confirmed cases'

    chart = uk_regions_trend_dashboard_charts(df_time_series_england, filter_area_type='Upper tier local authority', normalised=True, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Lower Tier Authorities'
display_h1(slide_title)

if not include_uk_local_authorities:
    print('Not configured to show UK local authorities')
else:
    chart_height = 250
    trend_chart_width = 350
    totals_chart_width = 200

    footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} authorities by total confirmed cases'

    chart = uk_regions_trend_dashboard_charts(df_time_series_england, filter_area_type='Lower tier local authority', normalised=False, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Lower Tier Authorities per 100k Population'
display_h1(slide_title)

if not include_uk_local_authorities:
    print('Not configured to show UK local authorities')
else:
    chart_height = 250
    trend_chart_width = 350
    totals_chart_width = 200

    footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} authorities by total confirmed cases'

    chart = uk_regions_trend_dashboard_charts(df_time_series_england, filter_area_type='Lower tier local authority', normalised=True, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Covid-19 US'
display_h1(slide_title)
measure_daily = True
independent_y_scales = True

chart_width_large = 350
chart_width_small = 200
chart_height_large = 350
chart_height = 100

top_n = 20

def get_state_totals(df):
    df_totals = df.copy().drop(['country', 'type'], axis=1)
    df_totals = df_totals.groupby(['state']).agg({'lat':'mean', 'long':'mean', 'confirmed':'sum', 'deaths':'sum'})
    df_totals = df_totals.reset_index()
    
    return df_totals

def trend_chart_us(df, scale_type='linear', x_metric='date', metric='deaths', x_min=None, x_max=None, measure_daily=True, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()

    df[metric] = df[metric].diff().clip(lower=0)
    df = df.groupby(['country', 'date']).agg({'confirmed':'sum', 'deaths':'sum'})
    df = df.reset_index()
    df['weekAvg'] = df.groupby('country')[metric].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)

    y_max = df[metric].max()
    y_min = df[metric].min()
    if scale_type == 'log':
        y_min = max(y_min, 0.1)
        small_non_zero_value = 0.001
        df.loc[df['weekAvg']==0, 'weekAvg'] = small_non_zero_value # to enable log calculation without errors

    chart = alt.Chart().mark_area(
    ).encode(
        x   = 'date:T',
        y   = metric+':Q',
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip(metric+':Q', format=',d')]
    ).properties(height=chart_height, width=chart_width_large)

    line = alt.Chart().mark_line(
        color='red',
    ).encode(
        x='date:T',
        y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(type=scale_type, domain=[y_min, y_max], clamp=True)),
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    )

    chart = alt.layer(chart, line, data=df)
        
    chart = chart.properties(title=column_title)

    return chart

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_latest_daily_us(df, category='', metric='confirmed', row_title='', column_title=''):
    df = df.copy()

    df = df.copy().drop(['country', 'state', 'type'], axis=1)
    df = df.groupby(['date']).agg({'lat':'mean', 'long':'mean', 'confirmed':'sum', 'deaths':'sum'})
    df = df.reset_index()

    df[metric] = df[metric].diff().clip(lower=0) # delta
    if category:
        df = df.groupby([category]).last()
    else:
        df['scope'] = 'US'
        category = 'scope'
        df = df[-1:] # last value
    
    df = df.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    bars = alt.Chart(df).mark_bar(
    ).encode(
        x=f'{metric}:Q',
        y=alt.Y(category+':N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(category+':N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    text = bars.mark_text(
        align = 'right',
        baseline = 'middle',
        dx = -15,
        fontSize = 20,
#         color='black' # not working for some reason
    ).encode(
        color=alt.Color(f'{category}', legend=None, scale=alt.Scale(range=['white'])), # sets the whole bar colour for some reason
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.resolve_scale(color='independent')
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width_small)
    
    return chart

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_latest_daily_us_states(df, category='', sort_field='deaths', metric='confirmed', row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()
    
    df['deaths'] = df['deaths'].diff().clip(lower=0) # delta
    df['confirmed'] = df['confirmed'].diff().clip(lower=0) # delta

    df = df.groupby(['state', 'date']).agg({'confirmed':'sum', 'deaths':'sum'})
    df = df.reset_index()

    if category:
        df = df.groupby([category]).last()
        df = df.reset_index()
    else:
        df = df[-1:] # last value
        
    sort_order = None # initial value
    if sort_field and category:
        sort_order = list(df.sort_values(by=sort_field, ascending=False)[category].unique())

    df = df.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    bars = alt.Chart(df).mark_bar(
    ).transform_window(
        rank = 'rank({sort_field})',
        sort=[alt.SortField(sort_field, order='descending')]
    ).transform_filter(
        (alt.datum.rank < top_n)
    ).encode(
        x=f'{metric}:Q',
        y=alt.Y(category+':N', sort=sort_order, title=row_title), # sort='-x' to sort locally in this chart, or 'None' for no sort
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(category+':N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title, height=chart_height_large, width=chart_width_small)
    
    return chart

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_us_totals(df, category='', metric='confirmed', row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()
    df['metric'] = df[metric] # didn't find a way to successfully use 'metric' variable inside transform_aggregate
    df['scope'] = 'US'

    bars = alt.Chart(df).mark_bar(
    ).transform_aggregate(
        total = 'sum(metric)',
        groupby = ['scope']
    ).encode(
        x='total:Q',
        y=alt.Y('scope:N', title=row_title),
        tooltip=[alt.Tooltip('scope:N'),
                 alt.Tooltip('total:Q', format=',d')]
    )

    text = bars.mark_text(
        align = 'right',
        baseline = 'middle',
        dx = -15,
        color = 'white',
        fontSize=20
    ).encode(
        text=alt.Text('total:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width_small)
    
    return chart

def trend_chart_us_states(df, scale_type='linear', x_metric='date', metric='deaths', x_min=None, x_max=None, measure_daily=True, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()

    if measure_daily:
        df[metric] = df[metric].diff().clip(lower=0)
    x_metric_type = ':T' # for date
    if x_metric == 'days_since_100':
        df = df[df['days_since_100']>=0]
        x_metric_type = ':Q' # for count of days
        x_min = 0 # no need for negative x-axis
        
    df = df.groupby(['state', 'date']).agg({'confirmed':'sum', 'deaths':'sum'})
    df = df.reset_index()

    df['weekAvg'] = df.groupby('state')[metric].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)

    y_max = df[metric].max()
    y_min = df[metric].min()
    if scale_type == 'log':
        y_min = max(y_min, 0.1)
        small_non_zero_value = 0.001
        df.loc[df['weekAvg']==0, 'weekAvg'] = small_non_zero_value # to enable log calculation without errors

    chart = alt.Chart(data=df).mark_line(
    ).encode(
        x='date:T',
        y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(type=scale_type, domain=[y_min, y_max], clamp=True)),
        color=alt.Color('state:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('state:N'),
                 alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    ).properties(height=chart_height, width=chart_width_large)

    chart = chart.properties(title=column_title)

    return chart

###########

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart_coloured_us_states(df, category='state', sort_field='deaths', data_field='confirmed', row_title='', column_title=''):
    df = df.copy()

    sort_order = None # initial value
    if sort_field and category:
        sort_order = list(df.sort_values(by=sort_field, ascending=False)[category].unique())

    bars = alt.Chart(df).mark_bar(
    ).transform_window(
        rank = 'rank({sort_field})',
        sort=[alt.SortField(sort_field, order='descending')]
    ).transform_filter(
        (alt.datum.rank < top_n)
    ).encode(
        x=alt.X(data_field+':Q', title=data_field),
        y=alt.Y(f'{category}:N', sort=sort_order, title=row_title), # sort='-x' to sort locally in this chart, or 'None' for no sort
        color=alt.Color(f'{category}:N', sort=None, legend=None, scale=alt.Scale(scheme='category20')),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(data_field+':Q', format=',d')]
    ).properties(
        height=chart_height_large, width=chart_width_small,
#         title='Covid total '+data_field+', '+continent
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(data_field+':Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title)

    return chart

def configure_trend_chart_us(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    
    return chart

def world_chart_summary(df_covid_time_series, top_n = 20, sort_field = 'deaths', scale_type = 'linear'):
    df_totals = df_covid_time_series.copy().groupby(['country', 'state']).last()
    df_totals = df_totals.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    df_state_totals_absolute       = get_state_totals(df_totals)

    df = df_covid_time_series.copy()
    
    if not top_n is None:
        df_totals = df.copy().groupby(['state']).agg({'confirmed':'sum', 'deaths':'sum'}).reset_index()
        df_totals = df_totals.sort_values(by=sort_field, ascending=False)
        df_totals = df_totals[:top_n]
        state_top_n = df_totals['state']
        df_covid_time_series = df[df['state'].isin(state_top_n)] # would like to put it in the order of highest totals

    trend_confirmed_cases  = trend_chart_us(df_covid_time_series, scale_type=scale_type, metric='confirmed', row_title='US', column_title='Daily Confirmed Cases')
    trend_deaths           = trend_chart_us(df_covid_time_series, scale_type=scale_type, metric='deaths', column_title='Daily Deaths')
    us_total_confirmed_cases = get_bar_chart_us_totals(df_state_totals_absolute, metric='confirmed', row_title='', column_title='Total Confirmed Cases')
    us_total_deaths     = get_bar_chart_us_totals(df_state_totals_absolute, metric='deaths', row_title='', column_title='Total Deaths')
    world_trend_row = trend_confirmed_cases | trend_deaths | us_total_confirmed_cases | us_total_deaths
    
    trend_confirmed_cases  = trend_chart_us_states(df_covid_time_series, scale_type=scale_type, metric='confirmed', row_title='States', column_title='Daily Confirmed Cases')
    trend_deaths           = trend_chart_us_states(df_covid_time_series, scale_type=scale_type, metric='deaths', row_title='', column_title='Daily Deaths')
    latest_confirmed_cases = get_bar_chart_latest_daily_us(df_covid_time_series, metric='confirmed', row_title='', column_title='Latest Daily Cases')
    latest_deaths          = get_bar_chart_latest_daily_us(df_covid_time_series, metric='deaths', row_title='', column_title='Latest Daily Deaths')
    continent_trend_row = trend_confirmed_cases | trend_deaths | latest_confirmed_cases | latest_deaths
    
    state_total_confirmed_cases  = get_bar_chart_coloured_us_states(df_state_totals_absolute, category='state', data_field='confirmed', sort_field='deaths', row_title='', column_title='Total Confirmed Cases')
    state_total_confirmed_deaths = get_bar_chart_coloured_us_states(df_state_totals_absolute, category='state', data_field='deaths', sort_field='deaths', row_title='', column_title='Total Deaths')
    latest_confirmed_cases = get_bar_chart_latest_daily_us_states(df_covid_time_series, category='state', metric='confirmed', sort_field='deaths', row_title='', column_title='Latest Daily Cases')
    latest_deaths          = get_bar_chart_latest_daily_us_states(df_covid_time_series, category='state', metric='deaths', sort_field='deaths', row_title='', column_title='Latest Daily Deaths')

    totals_row = state_total_confirmed_cases | state_total_confirmed_deaths | latest_confirmed_cases | latest_deaths

    
    chart = world_trend_row & continent_trend_row & totals_row

#     chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart_us(chart)
    
    return chart
    
footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} states by total deaths'

chart = world_chart_summary(df_covid_time_series_us, top_n = top_n)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
# slide_title = 'Covid-19 US - Log'
# display_h1(slide_title)

# footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} states by deaths'

# chart = world_chart_summary(df_covid_time_series_us, top_n = top_n, scale_type = 'log')
# chart.display()
# print(footer)

# if export_slides:
#     chart.save(saved_image_name)

#     # Add a new slide ready for chart and set title text
#     slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
#     prs.set_text(slide, placeholder_name='title', text=slide_title)
#     prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
#     prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Covid-19 US - States'
display_h1(slide_title)

chart_width_small = 120
chart_height = 120

top_n = 20

def trend_chart_us_states(df, scale_type='linear', x_metric='date', metric='deaths', sort_field='deaths', x_min=None, x_max=None, measure_daily=True, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()

    if measure_daily:
        df[metric] = df[metric].diff().clip(lower=0)
    x_metric_type = ':T' # for date
    if x_metric == 'days_since_100':
        df = df[df['days_since_100']>=0]
        x_metric_type = ':Q' # for count of days
        x_min = 0 # no need for negative x-axis
        
    df = df.groupby(['state', 'date']).agg({'confirmed':'sum', 'deaths':'sum'})
    df = df.reset_index()

    df['weekAvg'] = df.groupby('state')[metric].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
    
    y_max = df[metric].max()
    y_min = df[metric].min()
    if scale_type == 'log':
        y_min = max(y_min, 0.1)
        small_non_zero_value = 0.001
        df.loc[df['weekAvg']==0, 'weekAvg'] = small_non_zero_value # to enable log calculation without errors

    chart = alt.Chart().mark_area(
    ).encode(
        x   = alt.X('date:T', title=None),
        y   = metric+':Q',
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip(metric+':Q', format=',d')]
    ).properties(height=chart_height, width=chart_width_small)

    line = alt.Chart().mark_line(
        color='red',
    ).encode(
        x='date:T',
#         y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(type=scale_type, domain=[y_min, y_max], clamp=True)),
        y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(type=scale_type, clamp=True)),
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    )

    chart = alt.layer(chart, line, data=df).facet(alt.Column('state:N', title=None), columns=5).resolve_scale(y='independent')
        
    chart = chart.properties(title=column_title)

    return chart

def configure_trend_chart_us_states(chart):
    chart = chart.configure_headerRow(
        title=None,
    )
    
    chart = chart.configure_header(
        labelFontSize=16
    )

    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    
    return chart

def us_states_chart_summary(df, top_n = 20, sort_field = 'deaths', scale_type = 'linear'):
    df = df.copy()
    
    if not top_n is None:
        df_totals = df.copy().groupby(['state']).agg({'confirmed':'sum', 'deaths':'sum'}).reset_index()
        df_totals = df_totals.sort_values(by=sort_field, ascending=False)
        df_totals = df_totals[:top_n]
        state_top_n = df_totals['state']
        df = df[df['state'].isin(state_top_n)] # would like to put it in the order of highest totals

    us_states_trend_confirmed = trend_chart_us_states(df, scale_type=scale_type, metric='confirmed', sort_field='confirmed', row_title='', column_title='Daily Confirmed Cases')
    us_states_trend_deaths    = trend_chart_us_states(df, scale_type=scale_type, metric='deaths', sort_field='deaths', row_title='', column_title='Daily Deaths')

    chart = alt.hconcat(us_states_trend_confirmed, us_states_trend_deaths).configure_concat(spacing=100)
    chart = configure_trend_chart_us_states(chart)
    
    return chart
    
footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} states by total deaths'

chart = us_states_chart_summary(df_covid_time_series_us, top_n = top_n, sort_field = 'deaths')
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Continents'
display_h1(slide_title)

# category: 'continent', 'country' or 'Combined_Key'; data_field: 'confirmed' or 'deaths'
def get_bar_chart(df, category='continent', data_field='confirmed', row_title='', column_title=''):
    bars = alt.Chart(df).mark_bar().encode(
        x=data_field+':Q',
        y=alt.Y(category+':N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
        tooltip=[alt.Tooltip(category+':N'),
                 alt.Tooltip(data_field+':Q', format=',d')]
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(data_field+':Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title)
    
    return chart

def configure_bar_chart(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )

    chart = chart.configure_title(
        fontSize=20,
        color='blue'
    )
    return chart

df_continent_totals_absolute       = get_continent_totals(df_totals)
df_continent_totals_per_population = get_continent_totals(df_totals)

sort_by_confirmed = False # sort by confirmed cases or deaths
sort_field_absolute = 'confirmed' if sort_by_confirmed else 'deaths'
sort_field_per_population = 'confirmed_per_M_population' if sort_by_confirmed else 'deaths_per_M_population'

print(f'Totals sorted by {sort_field_absolute}; '+latest_date_str)
df_continent_totals_absolute = df_continent_totals_absolute.sort_values(by=sort_field_absolute, ascending=False)
df_continent_totals_per_population = df_continent_totals_per_population.sort_values(by=sort_field_per_population, ascending=False)

chart_confirmed = get_bar_chart(df_continent_totals_absolute, category='continent', data_field='confirmed', row_title='Absolute', column_title='Confirmed Cases')
chart_deaths    = get_bar_chart(df_continent_totals_absolute, category='continent', data_field='deaths', column_title='Deaths')
chart_population    = get_bar_chart(df_continent_totals_absolute, category='continent', data_field='Population', column_title='Population')
row_absolute = chart_confirmed | chart_deaths | chart_population

chart_confirmed = get_bar_chart(df_continent_totals_per_population, category='continent', data_field='confirmed_per_M_population', row_title='Per M Population')
chart_deaths    = get_bar_chart(df_continent_totals_per_population, category='continent', data_field='deaths_per_M_population')
chart_population    = get_bar_chart(df_continent_totals_per_population, category='continent', data_field='Population')
row_per_population = chart_confirmed | chart_deaths | chart_population

chart = (row_absolute & row_per_population).resolve_scale(color='independent').configure_legend(title=None, labelFontSize=11, symbolSize=15)
chart = configure_bar_chart(chart)

chart.display()

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)

In [ ]:
slide_title = 'Countries'
display_h1(slide_title)
top_n = 20
sort_by_confirmed = False # sort by confirmed cases or deaths

sort_field_absolute = 'confirmed' if sort_by_confirmed else 'deaths'
sort_field_per_population = 'confirmed_per_M_population' if sort_by_confirmed else 'deaths_per_M_population'

footer = f'Totals, top {top_n} by {sort_field_absolute}'
print(footer)

df_totals_filtered_absolute       = df_totals.copy()
df_totals_filtered_per_population = df_totals.copy()

df_totals_filtered_absolute       = df_totals_filtered_absolute.sort_values(by=sort_field_absolute, ascending=False)
df_totals_filtered_per_population = df_totals_filtered_per_population.sort_values(by=sort_field_per_population, ascending=False)

df_totals_filtered_absolute = df_totals_filtered_absolute[:top_n]
df_totals_filtered_per_population = df_totals_filtered_per_population[:top_n]

chart_confirmed = get_bar_chart(df_totals_filtered_absolute, category='Combined_Key', data_field='confirmed', row_title='Absolute', column_title='Confirmed Cases')
chart_deaths    = get_bar_chart(df_totals_filtered_absolute, category='Combined_Key', data_field='deaths', column_title='Deaths')
chart_population    = get_bar_chart(df_totals_filtered_absolute, category='Combined_Key', data_field='Population', column_title='Population')
row_absolute = chart_confirmed | chart_deaths | chart_population

chart_confirmed = get_bar_chart(df_totals_filtered_per_population, category='Combined_Key', data_field='confirmed_per_M_population', row_title='Per M Population')
chart_deaths    = get_bar_chart(df_totals_filtered_per_population, category='Combined_Key', data_field='deaths_per_M_population')
chart_population    = get_bar_chart(df_totals_filtered_per_population, category='Combined_Key', data_field='Population')
row_per_population = chart_confirmed | chart_deaths | chart_population

chart = (row_absolute & row_per_population).resolve_scale(color='independent').configure_legend(title=None, labelFontSize=11, symbolSize=15)

chart = configure_bar_chart(chart)

chart.display()

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
top_n = 10

slide_title = 'COVID-19 Trend - Per M Population'
footer = f'Per continent, top {top_n} by total {sort_field_absolute}; trend charts using 7 day rolling mean'
display_h2(slide_title)

chart = trend_dashboard_charts(df_covid_time_series, df_totals, sort_field=sort_field_per_population, confirmed_field='confirmed_per_M_population', deaths_field='deaths_per_M_population')
chart = configure_trend_chart_continents(chart)

chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
top_n = 10

slide_title = 'COVID-19 Trend - Log Absolute Since 100 cases'
footer = f'Per continent, top {top_n} by total {sort_field_absolute}; trend charts using 7 day rolling mean; y-axis minimum range clipped at 0.1'

display_h2(slide_title)

chart = trend_dashboard_charts(df_covid_time_series, df_totals, sort_field=sort_field_absolute, confirmed_field='confirmed', deaths_field='deaths', scale_type='log', x_metric='days_since_100')
chart = configure_trend_chart_continents(chart)

chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

slide_title = 'COVID-19 Trend - Log Per M Population Since 100 cases'
footer = f'Per continent, top {top_n} by total {sort_field_absolute}; trend charts using 7 day rolling mean; y-axis minimum range clipped at 0.1'
display_h2(slide_title)

chart = trend_dashboard_charts(df_covid_time_series, df_totals, sort_field=sort_field_per_population, confirmed_field='confirmed_per_M_population', deaths_field='deaths_per_M_population', scale_type='log', x_metric='days_since_100')
chart = configure_trend_chart_continents(chart)

chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Trend Charts - Selected Countries'
display_h1(slide_title)
small_subset = True
measure_daily = True
independent_y_scales = True

if small_subset:
    Combined_Key_list = ['India', 'Brazil', 'Sweden', 'United Kingdom', 'US', 'Ireland']

    print('Country list:', Combined_Key_list, latest_date_str)
    df_covid_time_series_filtered = df_covid_time_series[df_covid_time_series['Combined_Key'].isin(Combined_Key_list)].copy()
else:
    df_covid_time_series_filtered = df_covid_time_series.copy()

def trend_chart(df, metric='deaths', measure_daily=True, column_title=''):
    df = df.copy()
    df = df.reset_index()

    if measure_daily:
        df[metric] = df[metric].diff().clip(lower=0)

    chart = alt.Chart().mark_area().encode(
        x   = 'date:T',
        y   = metric+':Q',
#         row = alt.Row('country:N'),
        tooltip=[alt.Tooltip('Combined_Key:N'),
                 alt.Tooltip('date:T'),
                 alt.Tooltip(metric+':Q', format=',d')]
    ).properties(height=100, width=500)
    
    line = alt.Chart().mark_line(
        color='red',
    ).transform_window(
        weekAvg=f'mean({metric})',
        frame=[-7, 0],
        groupby=['country']
    ).encode(
        x='date:T',
        y='weekAvg:Q',
        tooltip=[alt.Tooltip('Combined_Key:N'),
                 alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    )
    
    chart = alt.layer(chart, line, data=df).facet(row='Combined_Key:N')
    if independent_y_scales:
        chart = chart.resolve_scale(y='independent')#.properties(height=50, width=400)

    chart = chart.properties(title=column_title)

    return chart

def configure_trend_chart(chart):
    chart = chart.configure_headerRow(
        title=None,
        labelColor='blue',
        labelFontSize=20
    )

    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    return chart

footer = 'Blue area chart is daily data; red line is 7 day rolling mean'

trend_confirmed_cases = trend_chart(df_covid_time_series_filtered, metric='confirmed', measure_daily=measure_daily, column_title='Daily Confirmed Cases')
trend_deaths          = trend_chart(df_covid_time_series_filtered, metric='deaths', measure_daily=measure_daily, column_title='Daily Deaths')
chart = trend_confirmed_cases | trend_deaths
chart = configure_trend_chart(chart)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
display_h1('Maps')
# countries = alt.topo_feature(data.world_110m.url, 'countries')
# Problem:
#   Country code 250 appears to be used twice in the map data
#   both for France, and French Guiana
#   patched in local edited world-110m_edited.json file 

filepath = 'world-110m_edited.json'
with open(filepath) as json_data:
    d = json.load(json_data)
        
    countries_map_data = alt.Data(
        values=d, 
        format=alt.TopoDataFormat(feature='countries',type='topojson')
    )

country_code_url = 'https://raw.githubusercontent.com/alisle/world-110m-country-codes/master/world-110m-country-codes.json'
country_code_df = pd.read_json(country_code_url)

In [ ]:
display_h2('World COVID Cases')
print(latest_date_str)
width = 700
aspect_ratio = 750/450

def map_covid(metric='deaths', project='equirectangular', width=800, aspect_ratio=750/450, clip_europe=False):
    world_map = alt.Chart(countries_map_data).mark_geoshape(
        fill='#666666',
        stroke='white'
    ).encode(
        tooltip=['name:N']
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(country_code_df, 'id', ['name'])
    ).properties(
        title='Covid '+metric,
        width=width,
        height=width/aspect_ratio
    ).project(project)
    
    # covid positions on map
    points = alt.Chart(df_totals).mark_circle().encode(
        longitude='long:Q',
        latitude='lat:Q',
        size=alt.Size(metric+':Q', title='Number of '+metric),
        color=alt.value('red'), #steelblue
        tooltip=[alt.Tooltip('country:N'),
                 alt.Tooltip(metric+':Q', format=',d')]
    ).properties(
        title='Number of covid '+metric,
    )
    
    if clip_europe:
        world_map = world_map.project(
            type= project,
            scale= 350,                          # Magnify
            center= [20,50],                     # [lon, lat]
            clipExtent= [[0, 0], [width, width/aspect_ratio]],    # [[left, top], [right, bottom]]
        )
        points = points.project(
            type= project,
            scale= 350,                          # Magnify
            center= [20,50],                     # [lon, lat]
            clipExtent= [[0, 0], [width, width/aspect_ratio]],    # [[left, top], [right, bottom]]
        )

    return world_map + points

world_covid_deaths = map_covid(metric='deaths', project='equirectangular', width=width, aspect_ratio=aspect_ratio)
world_covid_cases = map_covid(metric='confirmed', project='equirectangular', width=width, aspect_ratio=aspect_ratio)

chart= alt.hconcat(world_covid_cases, world_covid_deaths).resolve_scale(size='independent')

chart.display()

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text='World COVID Cases')
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)

In [ ]:
display_h2('Europe COVID Cases')
print(latest_date_str)

width = 400
aspect_ratio = 4/3

europe_covid_deaths = map_covid(metric='deaths', project='mercator', width=width, aspect_ratio=aspect_ratio, clip_europe=True)
europe_covid_cases = map_covid(metric='confirmed', project='mercator', width=width, aspect_ratio=aspect_ratio, clip_europe=True)

# alt.hconcat(europe_covid_cases, europe_covid_deaths).resolve_scale(size='independent').display() # displays with different center position for some reasons
europe_covid_cases.display()
europe_covid_deaths.display()

In [ ]:
display_h2('World COVID Cases - alternative map projections')
print(latest_date_str)

width = 500
aspect_ratio = 4/3

projections = ['equirectangular', 'mercator', 'orthographic'] # , 'gnomonic'

def project(chart, proj, width):
    chart = chart.project(proj).properties(title=proj, width=width, height=width/aspect_ratio)
    return chart

charts = [map_covid(metric='deaths', project=proj, width=width, aspect_ratio=aspect_ratio).properties(title=proj)
          for proj in projections]
alt.concat(*charts, columns=2)

In [ ]:
display_h1('Full country data')
display_h2('Bar chars - confirmed cases and deaths')

if not show_charts_for_all_countries:
    print('Not selected to show full country data')
else:
    print(latest_date_str)
    chart_confirmed = get_bar_chart(df_totals, category='Combined_Key', data_field='confirmed')
    chart_deaths    = get_bar_chart(df_totals, category='Combined_Key', data_field='deaths')

    chart_confirmed | chart_deaths

In [ ]:
display_h2('Trend charts')

if not show_charts_for_all_countries:
    print('Not selected to show full country data')
else:
    print(latest_date_str)
    df = df_covid_time_series.copy()
    measure_daily = True

    for country in df['Combined_Key'].unique():
        print(country)
        df_filtered = df[df['Combined_Key']==country].copy()

        trend_confirmed_cases = trend_chart(df_filtered, metric='confirmed', measure_daily=measure_daily)
        trend_deaths = trend_chart(df_filtered, metric='deaths', measure_daily=measure_daily)
        #     alt.hconcat(trend_confirmed_cases, trend_deaths).resolve_scale(size='independent').display()
        chart = trend_confirmed_cases | trend_deaths

        chart.display()

In [ ]:
current_year = 2020
clip_positive_only = True

# load country information
# https://www.mortality.org/
# https://www.mortality.org/Public/STMF_DOC/STMFNote.pdf
# https://www.mortality.org/Public/HMD-countries-codes.pdf
def load_short_term_mortality_timeseries_detailed():
    url = 'https://www.mortality.org/Public/STMF/Outputs/stmf.csv'
    df = pd.read_csv(url)
    
    # Drop redundant first row and set column names
    df = df.reset_index()
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])

    # Set data types
    df['Week'] = df['Week'].astype(int)
    df['Year'] = df['Year'].astype(int)
    df['DTotal'] = df['DTotal'].astype(float).astype(int)
    df[['D0_14', 'D15_64', 'D65_74', 'D75_84', 'D85p']] = df[['D0_14', 'D15_64', 'D65_74', 'D75_84', 'D85p']].astype(float)
    df['RTotal'] = df['RTotal'].astype(float)

    # - UK England & Wales Total Population GBRTENW
    # - DE Total population DEUTNP
    country_code_replace_dict = {'DEUTNP': 'DEU', 'GBRTENW': 'GBR', 'FRATNP': 'FRA'}
    df['CountryCode'] = df['CountryCode'].replace(country_code_replace_dict)
    country_code_to_name_dict = pycountry_convert.map_country_alpha3_to_country_name()
    country_code_to_name_dict['GBR_SCO'] = 'Scotland'
    df['Country'] = df['CountryCode'].map(country_code_to_name_dict)
    
    country_unknown = df[df['Country'].isnull()]['CountryCode'].drop_duplicates()
    if len(country_unknown):
        print('Unknown country codes:', country_unknown)

    return df

def load_short_term_mortality_data_detailed():
    df = load_short_term_mortality_timeseries_detailed()
    
#     df = add_continent(df)
    
#     df = df.set_index(['Continent', 'Country', 'Year', 'Week', 'Sex'])
    df = df.set_index(['Country', 'Year', 'Week', 'Sex'])
    df = df[~(df.index.get_level_values('Sex')=='b')] # drop the combined figure for both sexes
    
    df = df.filter(['D0_14', 'D15_64', 'D65_74', 'D75_84', 'D85p'])
    df = df.rename(columns={'D0_14':'Age 0-14', 'D15_64':'Age 15-64', 'D65_74':'Age 65-74', 'D75_84':'Age 75-84', 'D85p':'Age 85+'})

    df = df.reset_index()
#     df = pd.melt(df, id_vars=['Continent', 'Country', 'Year', 'Week', 'Sex'], var_name='AgeRange', value_name='Value')
    df = pd.melt(df, id_vars=['Country', 'Year', 'Week', 'Sex'], var_name='AgeRange', value_name='Value')
#     df = df.set_index(['Continent', 'Country', 'Year', 'Week', 'Sex', 'AgeRange'])
    df = df.set_index(['Country', 'Year', 'Week', 'Sex', 'AgeRange'])
    
    df_duplicated = df.loc[df.index.duplicated(keep=False)]
    if len(df_duplicated)>0:
        print('Duplicated index rows:')
        print(df_duplicated)
#         df = df.loc[~df.index.duplicated(keep='first')] # remove any duplicate index rows (e.g. for Vela at multiple frequencies)

    df = df.unstack(level=['AgeRange', 'Sex'])
    df = df.sort_index(axis='columns', level='AgeRange')

    return df

def get_excess_deaths_detailed(df, current_year, clip_positive_only):
    df = df.copy()
    
    df_previous_years = df.copy()
    df_previous_years = df_previous_years[df_previous_years.index.get_level_values('Year')<current_year]
    df_previous_years = df_previous_years.groupby(level=df_previous_years.index.names.difference(['Year']), axis=0).mean()

    df_excess_deaths = df.copy()
    df_excess_deaths = df_excess_deaths[df_excess_deaths.index.get_level_values('Year')==current_year]
    df_excess_deaths = df_excess_deaths.reset_index(level='Year', drop=True) # don't need this any more
    df_excess_deaths = df_excess_deaths - df_previous_years
    
    df_excess_deaths = df_excess_deaths.clip(lower=0) if clip_positive_only else df_excess_deaths
    
    return df_excess_deaths

def get_totals_from_time_series_detailed(df):
#     df_totals = df.copy().groupby(['Continent', 'Country']).sum()
    df_totals = df.copy().groupby(['Country']).sum()

    return df_totals

display_h1('Mortality Analysis')

df_stmf_detailed = load_short_term_mortality_data_detailed()
df_excess_deaths_detailed = get_excess_deaths_detailed(df_stmf_detailed, current_year=current_year, clip_positive_only=clip_positive_only)
df_excess_deaths_totals_detailed = get_totals_from_time_series_detailed(df_excess_deaths_detailed)

# display(df_stmf_detailed.head())

In [ ]:
independent_y_scales = True
chart_height = 100
chart_width = 200
current_year = 2020
clip_positive_only = True

def trend_chart_deaths_overall(df, current_year, scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
    df = df.copy()
    
    week_max = df[df['Year']==current_year]['Week'].max()
    week_trust_margin = 3
    week_num_trust = max(week_max-week_trust_margin, 0)
    
    y_min = 0 # max(df[metric].min(), 0)
    y_max = df[metric].max()
    
    line = alt.Chart().mark_line( #mark_area
    ).transform_filter(
        alt.datum.Year < 2020
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'mean({metric}):Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'mean({metric}):Q', format=',d')]
    )
    
    line_2020 = alt.Chart().mark_line(
        strokeDash=[5,5],
    ).transform_filter(
        (alt.datum.Year == 2020) &
#         (alt.datum.Sex == sex) & 
        (alt.datum.Week <= week_num_trust),
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'mean({metric}):Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('Week:Q'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    line_2020_recent = alt.Chart().mark_line(
        strokeDash=[1,3],
    ).transform_filter(
        (alt.datum.Year == 2020) &
#         (alt.datum.Sex == sex) & 
        (alt.datum.Week >= week_num_trust),
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'mean({metric}):Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('Week:Q'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    band = alt.Chart().mark_area(
        opacity = 0.3,
    ).transform_filter(
        alt.datum.Year < 2020
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'min({metric}):Q'),
        y2=alt.Y2(f'max({metric}):Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('Week:Q'),
                 alt.Tooltip(f'max({metric}):Q', title='Max', format=',d'),
                 alt.Tooltip(f'mean({metric}):Q', title='Mean', format=',d'),
                 alt.Tooltip(f'min({metric}):Q', title='Min', format=',d')]
    )
    
    chart = alt.layer(line, band, line_2020, line_2020_recent, data=df)
    chart = chart.properties(title=column_title, height=row_height, width=chart_width)

    return chart

def trend_chart_deaths_age_range(df, current_year, category='Country', sex='', scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
    df = df.copy()
    df = df.reset_index()
    
    week_max = df[df['Year']==current_year]['Week'].max()
    week_trust_margin = 3
    week_num_trust = max(week_max-week_trust_margin, 0)
    
    y_min = max(df[metric].min(), 0)
    y_max = df[metric].max()
    
    line = alt.Chart().mark_line( #mark_area
    ).transform_filter(
        alt.datum.Year < 2020
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'mean({metric}):Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'mean({metric}):Q', format=',d')]
    )
    
    line_2020 = alt.Chart().mark_line(
        strokeDash=[5,5],
    ).transform_filter(
        (alt.datum.Year == 2020) &
        (alt.datum.Sex == sex) & 
        (alt.datum.Week <= week_num_trust),
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'mean({metric}):Q'),
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip('Week:Q'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    line_2020_recent = alt.Chart().mark_line(
        strokeDash=[1,3],
    ).transform_filter(
        (alt.datum.Year == 2020) &
        (alt.datum.Sex == sex) & 
        (alt.datum.Week >= week_num_trust),
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'mean({metric}):Q'),
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip('Week:Q'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    band = alt.Chart().mark_area(
        opacity = 0.3,
    ).transform_filter(
        alt.datum.Year < 2020
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'min({metric}):Q'),
        y2=alt.Y2(f'max({metric}):Q'),
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip('Week:Q'),
                 alt.Tooltip(f'max({metric}):Q', title='Max', format=',d'),
                 alt.Tooltip(f'mean({metric}):Q', title='Mean', format=',d'),
                 alt.Tooltip(f'min({metric}):Q', title='Min', format=',d')]
    )
    
    chart = alt.layer(line, band, line_2020, line_2020_recent, data=df)
    chart = chart.properties(title=column_title, height=row_height, width=chart_width)

    return chart

def trend_chart_excess_deaths_age_range(df, category='Country', sex='', scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
    df = df.copy()
    df = df.reset_index()
    
    week_max = df['Week'].max()
    week_trust_margin = 3
    week_num_trust = max(week_max-week_trust_margin, 0)

    y_min = df[metric].min()
    y_max = df[metric].max()
    
    line = alt.Chart().mark_line(
        strokeDash=[5,5],
    ).transform_filter(
        (alt.datum.Sex == sex) & 
        (alt.datum.Week <= week_num_trust),
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'{metric}:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )
        
    line_recent = alt.Chart().mark_line(
        strokeDash=[1,3],
    ).transform_filter(
        (alt.datum.Sex == sex) & 
        (alt.datum.Week >= week_num_trust),
    ).encode(
        x=alt.X('Week:Q'),
        y=alt.Y(f'{metric}:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    chart = alt.layer(line, line_recent, data=df)
    chart = chart.properties(title=column_title, height=row_height, width=chart_width)

    return chart

# category: 'continent', 'country'
def get_bar_chart_coloured_age_range(df, category='Country', sex='', scale_type='linear', metric='DTotal', row_title='', column_title='', x_min=None, x_max=None, row_height=200, chart_width=200):
    x_min = 0 if not x_min else x_min
    x_max = df[metric].max() if not x_max else x_max
    
    bars = alt.Chart(df).mark_bar().transform_filter(
        (alt.datum.Sex == sex)
    ).encode(
        x=alt.X(f'{metric}:Q', title=metric, scale=alt.Scale(domain=[x_min, x_max], type=scale_type)),
        y=alt.Y(f'{category}:N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    ).properties(
        height=row_height, width=chart_width,
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(metric+':Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title)

    return chart

# category: 'continent', 'country'
def get_bar_chart_coloured_overall(df, scale_type='linear', metric='DTotal', row_title='', column_title='', x_min=None, x_max=None, row_height=200, chart_width=200):
    x_min = 0 if not x_min else x_min
    x_max = df[metric].max() if not x_max else x_max
    
    bars = alt.Chart(df).mark_bar().encode(
        x=alt.X(f'{metric}:Q', title=metric, scale=alt.Scale(domain=[x_min, x_max], type=scale_type)),
#         y=alt.Y(category+':N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{metric}:Q', format=',d')]
    ).properties(
        height=row_height, width=chart_width,
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title)

    return chart

def trend_dashboard_charts_age_range(df, df_excess, df_totals, current_year, clip_positive_only, top_n=None, metric='Value', scale_type='linear', country_filter_list=None, row_height=200, chart_width=200):
    df_overall = df.copy()
    df_overall = df_overall.stack(['AgeRange', 'Sex'])
    df_overall = df_overall.groupby(level=df_overall.index.names.difference(['AgeRange', 'Sex']), axis=0).sum()
    df_overall = df_overall.reset_index()
    df_overall = df_overall[df_overall['Country'].isin(country_filter_list)] if country_filter_list else df_overall
#     df_overall = df_overall.sum(axis=0)
#     display(df_overall.head())
    
    df = df.copy()
    df = df.stack(['AgeRange', 'Sex'])
    df = df.reset_index()
    df = df[df['Country'].isin(country_filter_list)] if country_filter_list else df
    
    df_excess = df_excess.copy()
    df_excess = df_excess.stack(['AgeRange', 'Sex'])
    df_excess = df_excess.clip(lower=0) if clip_positive_only else df_excess
    df_excess = df_excess.reset_index()
    df_excess = df_excess[df_excess['Country'].isin(country_filter_list)] if country_filter_list else df_excess

    df_totals = df_totals
    df_totals = df_totals.stack(['AgeRange', 'Sex'])
    df_totals = df_totals.clip(lower=0) if clip_positive_only else df_totals
    df_totals_overall = df_totals.groupby(level=df_totals.index.names.difference(['AgeRange', 'Sex']), axis=0).sum()
    df_totals = df_totals.reset_index()
    df_totals_overall = df_totals_overall.reset_index()
    df_totals = df_totals[df_totals['Country'].isin(country_filter_list)] if country_filter_list else df_totals
    
    chart = alt.vconcat()
    first_row = True
    
    category='Country'
    
    excess_deaths_max_per_category = None #df_totals[metric].max()
    excess_deaths_overall_max = df_totals_overall[metric].max()

#     country_list = set(df['Country'].unique()).intersection(country_filter_list) if country_filter_list else df['Country'].unique()
    country_list = df['Country'].unique()
    for country in country_list:
        df_overall_filtered = df_overall[df_overall['Country']==country]
        df_filtered = df[df['Country']==country]
        df_excess_filtered = df_excess[df_excess['Country']==country]
        df_totals_filtered = df_totals[df_totals['Country']==country]
        df_totals_overall_filtered = df_totals_overall[df_totals_overall['Country']==country]

        trend_deaths_overall = trend_chart_deaths_overall(df_overall_filtered, current_year=current_year, metric=metric,
                                          row_title=country, column_title='Deaths' if first_row else '',
                                          row_height=row_height, chart_width=chart_width)

#         trend_deaths_male = trend_chart_deaths_age_range(df_filtered, current_year=current_year, category='AgeRange', sex='m', metric=metric,
#                                           row_title='', column_title='Deaths Male' if first_row else '',
#                                           row_height=row_height, chart_width=chart_width)

#         trend_deaths_female = trend_chart_deaths_age_range(df_filtered, current_year=current_year, category='AgeRange', sex='f', metric=metric,
#                                           row_title='', column_title='Deaths Female' if first_row else '',
#                                           row_height=row_height, chart_width=chart_width)

        trend_excess_deaths_male = trend_chart_excess_deaths_age_range(df_excess_filtered, category='AgeRange', sex='m', metric=metric,
                                                        row_title='', column_title='Excess Deaths Male' if first_row else '',
                                                        row_height=row_height, chart_width=chart_width)

        trend_excess_deaths_female = trend_chart_excess_deaths_age_range(df_excess_filtered, category='AgeRange', sex='f', metric=metric,
                                                        row_title='', column_title='Excess Deaths Female' if first_row else '',
                                                        row_height=row_height, chart_width=chart_width)

        chart_deaths_male    = get_bar_chart_coloured_age_range(df_totals_filtered, category='AgeRange', sex='m', metric=metric, 
                                                        row_title='', column_title='Total Excess Deaths Male' if first_row else '',
                                                        x_min=0, x_max=excess_deaths_max_per_category,
                                                        row_height=row_height, chart_width=chart_width)
        
        chart_deaths_female    = get_bar_chart_coloured_age_range(df_totals_filtered, category='AgeRange', sex='f', metric=metric, 
                                                        row_title='', column_title='Total Excess Deaths Female' if first_row else '',
                                                        x_min=0, x_max=excess_deaths_max_per_category,
                                                        row_height=row_height, chart_width=chart_width)

        chart_deaths_overall    = get_bar_chart_coloured_overall(df_totals_overall_filtered, metric=metric, 
                                                        row_title='', column_title='Total Excess Deaths' if first_row else '',
                                                        x_min=0, x_max=excess_deaths_overall_max,
                                                        row_height=row_height, chart_width=chart_width)

        first_row = False

#         row = trend_deaths_overall | trend_deaths_male | trend_deaths_female | trend_excess_deaths_male | trend_excess_deaths_female | chart_deaths_male | chart_deaths_female
        row = trend_deaths_overall | trend_excess_deaths_male | trend_excess_deaths_female | chart_deaths_male | chart_deaths_female | chart_deaths_overall
        chart = (chart & row).resolve_scale(color='independent')
#         chart = chart.configure_legend(title=None, labelFontSize=11, symbolSize=15)
#         chart = chart.configure_legend(orient='none') # disable legend since colours are already explained in bar chart
    
    return chart


def configure_trend_chart_continents(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    chart = chart.configure_axisX(
#         title=None
    )

    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    return chart

country_filter_list = ['Spain', 'Sweden', 'United Kingdom', 'United States']

slide_title = 'Excess Deaths '+str(current_year) + ' - Selected Countries'
display_h2(slide_title)
print('Dashed line is for current year; dotted is for last 3 weeks since data might not yet be final')
print('Shaded area is min/max range for previous years, with mean as solid line')
print('Where current year is significantly lower than previous years, it is likely to be due to data not yet available')

footer = 'Dashed line is current year; dotted is last 3wks; shaded is min/max for previous years, with mean as solid line'
chart = trend_dashboard_charts_age_range(df_stmf_detailed, df_excess_deaths_detailed, df_excess_deaths_totals_detailed, 
                                         current_year=current_year, clip_positive_only=clip_positive_only,
                                         country_filter_list=country_filter_list,
                                         row_height=chart_height, chart_width=chart_width)
chart = configure_trend_chart_continents(chart)

chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
if export_slides:
    # save slides
    print('Saving slides')
    prs.save(slides_file_name)